# Script to Crosswalk Upper Columbia Habitat Prioritization (Habitat Quality and Limiting Factors) from RTT  
# Prioritization Process and Reach Assessment Projects
## Author: Ryan Niemeyer, Upper Columbia Salmon Recovery Board
### For more information, see https://www.ucsrb.org/prioritization/


In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/ff-storage-p01/festivals/logos/000/051/750/large/logo.jpg?1575572027", width=200, height=200)

## ------------------------------------ Upload Python Libraries -----------------------------------

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# import geopandas as gpd
pd.options.display.max_colwidth = 150 # make it so print statements can be longer
pd.options.display.max_columns = None
pd.options.display.max_rows = 200
# CHEAT SHEET for pd and np commands: https://www.dataquest.io/blog/pandas-cheat-sheet/#:~:text=%20Pandas%20Cheat%20Sheet%20%E2%80%94%20Python%20for%20Data,CSV%2C%20.xlsx%2C%20SQL%2C%20or%20JSON.%20%20More%20
# to print entire data frame: pd.set_option('display.max_rows', None)

In [478]:
# import sys
# 'geopandas' in sys.modules
# !conda install --yes --prefix {sys.prefix} geopandas

In [55]:
# PART 1: do a crosswalk between the habitat attributes and the projects (via Action_CategoryandType_DRAFT09022020xlsx tabs)

# PART 2: generate projects for each reaches based on their habitat scores
# Step 1: Identify Priority Reaches (via HQ or LF) - pull up the reach (do this for all)
# Step 2: identify what habitat attributes are impaired based on a criteria (Habitat Quality or Limiting Factor?)
#           A) figure out what's going on wiht "Riparian Structure"
# Step 3: Identify Action Types/Categories linked to impacted Habitat Attributes in each impaired reach (based on criteria in Step 1)
# Step 4: Output Data


## ----------------------------------------------------------------------------------------------------------------------------
#             Choose the Criteria for Habitat Quality Output Table and Output  
## ----------------------------------------------------------------------------------------------------------------------------
### !!!!! DOUBLE CHECK these are the criteria you want !!!!!!!!!!!!!!

## Criteria for Query
### Update if criteria for output habitat quality tables change

In [615]:
# ---------- put the minimum allowable score -----------
# cut off value for actions to include for individual habitat attributes in Habitat Quality analysis
habitat_quality_priority_score = 5      # HQ script will pull HQ_score for habitat attributes with this OR higher HQ_score values

# -------- individual habitat score filter ---------
# Oct 2020: do 1 (Acceptable only)   OR 3 (Acceptable and At Risk) - does an equal and less than
indiv_habitat_attribute_score = 1           # for pulling actions JUST from - it's the minimum score - so put 1 or 3
# Note - this is redundant with the habitat_quality_priority_score:    limiting_factor_priority_score = 1

## Reach Confinment Criteria

In [616]:
# --------------- Reach Confinment (data in confinement_scores) --------------------------------
# 1, 3, 5, where 1 is higher % confined, 5 is higher % unconfined, based on USGS Valley Confinment Algorithm
# Habitat Quality analysis filters out based on confinement during intitial filters

Reach_Confinement_Limiting_Factors = {                  
    'Floodplain Reconnection': ['5'] }

# ------ Reach_Confinement: confinement_scores = 'Score' -------
Reach_Confinement_vars = { 
 'data_frame': ['confinement_scores' ], 
 'variable': ['Score'] 
} 
# create a dataframe 
Reach_Confinement_vars = pd.DataFrame(Reach_Confinement_vars, columns = ['data_frame', 'variable']) 


## ----------------------------------------------------------------------------------------------------------------------------
##             Output Directory
## ----------------------------------------------------------------------------------------------------------------------------
### Update if you want output path to change

In [617]:
# ------------------------ Output for Crosswalk between Habitat Attributes and Action Categories and Type
output_Crosswalk_Habitat_Attributes_and_Actions = 'Crosswalk_Habitat_Attributes_and_Actions.xlsx'
# ONLY for if someone else is using the original    output_Crosswalk_Habitat_Attributes_and_Actions   = 'Crosswalk_Habitat_Attributes_and_Actions_8Sept2020_Copy_for_Python_if_Original_being_used.xlsx'
output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/'
#output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/'
output_Crosswalk_Habitat_Attributes_and_Actions =  output_location + output_Crosswalk_Habitat_Attributes_and_Actions
print('output for Habitat Attributes and Actions List: ' )
print(output_Crosswalk_Habitat_Attributes_and_Actions)
print("    ")

# ------------------- Output for Project List based on Habitat Prioritization --------------------
output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/'
#output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/'

# ---------- Habitat Quality- Reach Assessment Projects Only -----------
output_name_Spring_Chinook_Habitat_Quality = 'Prioritized_Project_List_Habitat_Quality_Output_Spring_Chinook.xlsx'
output_name_Steelhead_Habitat_Quality  = 'Prioritized_Project_List_Habitat_Quality_Output_Steelhead.xlsx'
output_location_Spring_Chinook_Habitat_Quality_Projects_only =  output_location + output_name_Spring_Chinook_Habitat_Quality
output_location_Steelhead_Habitat_Quality_Projects_only =  output_location + output_name_Steelhead_Habitat_Quality

# ---------- Habitat Quality- Action Categories/Types for all impaired reaches, and Projects when present -----------
output_name_Spring_Chinook_Habitat_Quality_Actions = 'Impaired_Habitat_Reaches_Actions_List_Habitat_Quality_Output_Spring_Chinook.xlsx'
output_name_Steelhead_Habitat_Quality_Actions  = 'Impaired_Habitat_Reaches_Actions_Habitat_Quality_Output_Steelhead.xlsx'
output_location_Spring_Chinook_Habitat_Quality_All_Actions =  output_location + output_name_Spring_Chinook_Habitat_Quality_Actions
output_location_Steelhead_Habitat_Quality_All_Actions =  output_location + output_name_Steelhead_Habitat_Quality_Actions

# ---------- Limiting Factors - All Projects ------------
output_name_Spring_Chinook_Limiting_Factors = 'Prioritized_Project_List_Limiting_Factors_Output_Spring_Chinook.xlsx'
output_name_Steelhead_Limiting_Factors = 'Prioritized_Project_List_Limiting_Factors_Output_Steelhead.xlsx'
output_location_Spring_Chinook_Limiting_Factors_All_Projects =  output_location + output_name_Spring_Chinook_Limiting_Factors
output_location_Steelhead_Limiting_Factors_All_Projects =  output_location + output_name_Steelhead_Limiting_Factors

# ---------- Limiting Factors - all actions for impaired life stages ------------
output_name_Spring_Chinook_Limiting_Factors = 'Impaired_Habitat_Reaches_Actions_List_Limiting_Factors_Output_Spring_Chinook.xlsx'
output_name_Steelhead_Limiting_Factors = 'Impaired_Habitat_Reaches_Actions_List_Limiting_Factors_Output_Steelhead.xlsx'
output_location_Spring_Chinook_Limiting_Factors_All_Actions =  output_location + output_name_Spring_Chinook_Limiting_Factors
output_location_Steelhead_Limiting_Factors_All_Actions =  output_location + output_name_Steelhead_Limiting_Factors

# ---------------------- Output for large output file with all reaches and projects for habitat attributes for reaches ----
output_name_Spring_Chinook_Limiting_Factors = 'COMPLETE_Project_List_Limiting_Factors_Output_Spring_Chinook_ALL_reaches.xlsx'
output_name_Steelhead_Limiting_Factors = 'COMPLETE_Prioritized_Project_List_Limiting_Factors_Output_Steelhead_ALL_reaches.xlsx'
output_location_COMPLETE_Spring_Chinook_Limiting_Factors =  output_location + output_name_Spring_Chinook_Limiting_Factors
output_location_COMPLETE_Steelhead_Limiting_Factors =  output_location + output_name_Steelhead_Limiting_Factors



output for Habitat Attributes and Actions List: 
Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Crosswalk_Habitat_Attributes_and_Actions.xlsx
    


## ---------------------------------------------------------------------------------------------------------------------------
## Tables and Column Names of Indicators/Variables: 
### Update if excel input names change OR column names change
## ---------------------------------------------------------------------------------------------------------------------------

In [618]:
# -------------------- List the life stages --------------
life_stages_all = {                   'life_stages': ['Adult Migration','Holding and Maturation','Spawning','Fry','Summer Rearing',
                   'Winter Rearing','Smolt'] } 
# create data frame 
life_stages_all = pd.DataFrame(life_stages_all, columns = ['life_stages']) 

## ----------------------------------------------------------------------------------------------------------- 
##                     Upload the Habitat Data Excel
## ----------------------------------------------------------------------------------------------------------- 

In [619]:
# ----------- Upload tabs for table that is crosswalk between Habitat Attributes and Project Categories
Habitat_Attribute_Category_Data = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx',
                   'Attribute_Action Crosswalk')
Category_Type_Data = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Projects/Action_CategoryandType_DRAFT09022020xlsx.xlsx', 
                   'Cateogry_Type')

In [620]:
# -------------- Upload Project List, copied over from Reach Assessments -------------------------------------------------
Project_List_Data = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Projects/Updated_091120_ProjectTable_RAs.xlsx', 
                   'Data_Entry')

In [621]:
# ------------ remove any white space at beginning or end of attributes -----------------
Habitat_Attribute_Category_Data = Habitat_Attribute_Category_Data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# ------------ remove underscores and replace with white space (just for Action Categories that had underscore in name) -----------------
Project_List_Data = Project_List_Data.applymap(lambda x: x.replace("_"," ") if isinstance(x, str) else x)

In [622]:
# ------------------------------- Confinement Scores ----------------------
confinement_scores = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'Confinement_Scores')

In [623]:
# ------------------------------------------------------------------------------
#      Open Limiting Factor Data
# ------------------------------------------------------------------------------

# -----------------------  Loop to read in Limiting Factors for each Stages --------------------
# ---------- output directory ------------------------------
# NOTE: just need to change this directory, remaining script will generate pathways automatically
output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/'
#output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/'

# ------------------------ Spring Chinook output paths ---------------
species_x = "Spring_Chinook"
Output_Spring_Chinook_list = []
for i in range(life_stages_all.shape[0]):
    
    # ------- generate life stage name for file name -----
    life_stage_output_string = str(life_stages_all.loc[i][0]).replace(" ", "_")
    # --------- generate path for life stage output ----------
    pathx = output_location + "Limiting_Factor_Output," + species_x + ","  + life_stage_output_string + ".xlsx" # ORIGINAL - only for impaired reaches
 #   pathx = output_location + "Limiting_Factor_Output," + species_x + ",ALL_REACHES,"  + life_stage_output_string + ".xlsx" # for "All reaches scores"
    # -------- append directory path to list ------
    Output_Spring_Chinook_list.append(pathx)
    
# ------------------------- Read in Spring Chinook Limiting Factors ---------
Limiting_Factor_Spring_Chinook_Adult_Migration = pd.read_excel(Output_Spring_Chinook_list[0])
Limiting_Factor_Spring_Chinook_Holding_and_Maturation = pd.read_excel(Output_Spring_Chinook_list[1])
Limiting_Factor_Spring_Chinook_Spawning = pd.read_excel(Output_Spring_Chinook_list[2])
Limiting_Factor_Spring_Chinook_Fry = pd.read_excel(Output_Spring_Chinook_list[3])
Limiting_Factor_Spring_Chinook_Summer_Rearing = pd.read_excel(Output_Spring_Chinook_list[4])
Limiting_Factor_Spring_Chinook_Winter_Rearing = pd.read_excel(Output_Spring_Chinook_list[5])
Limiting_Factor_Spring_Chinook_Smolt = pd.read_excel(Output_Spring_Chinook_list[6])

# ------------------------ Steelhead output paths ---------------
species_x = "Steelhead"
Output_Steelhead_list = []
for i in range(life_stages_all.shape[0]):
    
    # ------- generate life stage name for file name -----
    life_stage_output_string = str(life_stages_all.loc[i][0]).replace(" ", "_")
    # --------- generate path for life stage output ----------
    pathx = output_location + "Limiting_Factor_Output," + species_x + ","  + life_stage_output_string + ".xlsx" # ORIGINAL
    #pathx = output_location + "Limiting_Factor_Output," + species_x + ",ALL_REACHES,"  + life_stage_output_string + ".xlsx" # for ALL reaches scores
    # -------- append directory path to list ------
    Output_Steelhead_list.append(pathx)
    
# ------------------------- Read in Steelhead Limiting Factors ---------
Limiting_Factor_Steelhead_Adult_Migration = pd.read_excel(Output_Steelhead_list[0])
Limiting_Factor_Steelhead_Holding_and_Maturation = pd.read_excel(Output_Steelhead_list[1])
Limiting_Factor_Steelhead_Spawning = pd.read_excel(Output_Steelhead_list[2])
Limiting_Factor_Steelhead_Fry = pd.read_excel(Output_Steelhead_list[3])
Limiting_Factor_Steelhead_Summer_Rearing = pd.read_excel(Output_Steelhead_list[4])
Limiting_Factor_Steelhead_Winter_Rearing = pd.read_excel(Output_Steelhead_list[5])
Limiting_Factor_Steelhead_Smolt = pd.read_excel(Output_Steelhead_list[6])

# -------------------- Open the Limiting Factor data that includes all life stages in one spreadsheet -----------
output_location_x = output_location + 'Limiting_Factor,Spring_Chinook,all_Life_Stages,ALL_SCORES.xlsx'
Limiting_Factor_Spring_Chinook_ALL_Reaches = pd.read_excel(output_location_x)
output_location_x = output_location + 'Limiting_Factor,Steelhead,all_Life_Stages,ALL_SCORES.xlsx'
Limiting_Factor_Steelhead_ALL_Reaches = pd.read_excel(output_location_x)


In [624]:
# Habitat_Attribute_Category_Data['Habitat Attribute'].unique()

In [625]:
# Habitat_Attribute_Category_Data[Habitat_Attribute_Category_Data['Habitat Attribute'] == 'Temperature- Adult Holding']

## ----------------------------------------------------------------------------------------------------------- 
##      Generate Crosswalk between Habitat Attributes in Habitat Quality Analysis and in Action Category Tables
## ----------------------------------------------------------------------------------------------------------- 

In [626]:
# ------ Crosswalk between Habitat Attributes listed in the Habitat Quality Analysis and in Action Category Crosswalks

# KEY FIRST value is habitat attribute from Habitat Quality or Limiting Factors pathway output, 
#          the SECOND value is the habitat attribute from the project data
#     EXAMPLE: 'Temperature': ['Temperature'] ,    the 'Temperature' is from the HQ or LF data
#                                              and the " ['Temperature']" is from the project data
Habitat_Attribute_Names_Crosswalk = { 
 
    'Bank Stability': ['Bank Stability'],
    'Channel Stability': ['Channel Stability'],
    'Stability': ['Stability'],
    'Cover- Wood': ['Cover- Wood'],
    'Cover- Boulders': ['Cover- Boulders'],
    'Cover- Undercut Banks': ['Cover- Undercut Banks'],
    'Flow- Summer Base Flow': ['Flow- Summer Base Flow'],
    '% Fines/Embeddedness': ['% Fines/Embeddedness'],
    'Harassment': ['Harassment'],
    #'Off-Channel Total': ['Off-Channel- Floodplain'],
    'Off-Channel- Floodplain': ['Off-Channel- Floodplain'],
    'Off-Channel- Side-Channels': ['Off-Channel- Side-Channels'],
    'Pools- All Pools': ['Pools- All Pools'],
    'Quality Pools': ['Quality Pools'],
    'Pool Quantity & Quality': ['Quality Pools'],
    'Pool Quantity& Quality': ['Quality Pools'],
    'Pool Quantity & Quality': ['Pool Quantity & Quality'],
    'Pool Quantity& Quality': ['Pool Quantity& Quality'],
    'Pools- Deep Pools': ['Pools- Deep Pools'],
    #'Riparian- Canopy Cover': ['Riparian'],
    #'Riparian- Structure': ['Riparian'], 
    #'Riparian-Disturbance': ['Riparian'], 
    #'Stability Total': ['Channel Stability'],
    'Coarse Substrate': ['Quality Substrate'],
    'Coarse Substrate': ['Coarse Substrate'],
    'Quality Substrate': ['Substrate'],
    'Quality Substrate': ['Quality Substrate'],
    # 'Temperature': ['Temperature- Adult Holding', 'Temperature- Adult Spawning','Temperature- BT Holding', 
    #               'Temperature- BT Rearing','Temperature- FMO', 'Temperature- Summer Rearing',
    #                'Temperature- Thermal Barriers', 'Temperature- Winter Rearing'],
    'Temperature': ['Temperature'],
    'Temperature- Rearing': ['Temperature'],
    'Temperature- Rearing': ['Temperature- Rearing'],
    'Riparian': ['Riparian'],
    'Riparian- Structure' : ['Riparian- Structure'],
    'Riparian-Disturbance': ['Riparian-Disturbance'],
    'Riparian- Canopy Cover': ['Riparian- Canopy Cover'],
    'Entrainment- Fry': ['Entrainment- Fry'],
    'Predators Fry': ['Predators Fry'],
    'Predators- Adult': ['Predators- Adult'],
    'Food- Food Web Resources': ['Food- Food Web Resources'], 
    'Contaminants': ['Contaminants'],
    'Temperature- Adult Holding': ['Temperature- Adult Holding'],
    'Predators- Juveniles': ['Predators- Juveniles'],
    'Flow- Scour': ['Flow- Scour'],
     'Icing': ['Icing'], 
    'Entrainment- Summer Rearing': ['Entrainment- Summer Rearing'],
    #'Total Riparian': ['Riparian'], # not using this anymore (I think it morphed to "Riparian") 
    # all these are habitat attributes not present in the Project data:
    'Cover- Undercut Banks': ['NA'],  'Temperature- Adult Spawning': ['NA'],
    'Temperature- BT Holding': ['NA'], 'Temperature- BT Rearing': ['NA'],'Temperature- FMO': ['NA'], 
    'Temperature- Summer Rearing': ['NA'],'Temperature- Thermal Barriers': ['NA'], 'Temperature- Winter Rearing': ['NA'],
    'Harassment': ['NA'],  
    'Pool Riffle': ['NA'],  'Substrate- Gravel/Cobble': ['NA'],
    'Edge Habitat': ['NA'], 
    'Substrate- Diversity': ['NA']
} 

Habitat_Attribute_Names_Crosswalk_df = pd.DataFrame(Habitat_Attribute_Names_Crosswalk)

# -------------- list of habitat attributes from Habitat Quality output -----------
habitat_attributes = ['Temperature- Rearing',
       'Flow- Summer Base Flow', 'Riparian- Structure', 'Riparian-Disturbance', 'Riparian- Canopy Cover', 'Riparian',
        'Coarse Substrate', 'Cover- Wood','Pool Quantity& Quality', 'Off-Channel- Floodplain',
       'Off-Channel- Side-Channels', 'Channel Stability', 'Bank Stability', 'Stability']

# Habitat Attrbutes in Limiting Factors note included: '% Fines/Embeddedness', 'Cover- Boulders', 'Cover- Undercut Banks',
#                                                     'Contaminants', 'Substrate- Diversity', 'Edge Habitat', 'Pool Riffle'

In [627]:
# Habitat_Attribute_Category_Data[Habitat_Attribute_Category_Data['Habitat Attribute']=='Pools- Deep Pools']

## ----------------------------------------------------------------------------------------------------------------------------
#
#             Part 1: List Habitat Attributes for each Project Category
#
## ----------------------------------------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------------------------------------
##            Generate CrossWalk between Habitat Attributes for each Project Category
## ----------------------------------------------------------------------------------------------------------------------------

In [628]:
# ----------- Remove Action Types/Categories with no Action Type ---------------
Category_Type_Data_Updated = Category_Type_Data[~Category_Type_Data['Action Type'].isin(['?'])]

In [629]:
# ----------------------- Create Data Frame ---------------------
Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame = pd.DataFrame([])

# ------------------- Loop through each Action Type to get Habitat Attributes ----------
for action_type_x in Category_Type_Data_Updated['Action Type']:
    
    # ----------------- Identify the Action Category for this Action Type ------------
    action_category_x = Category_Type_Data_Updated[Category_Type_Data_Updated['Action Type'].isin([action_type_x])]['Action Category']

    # -------------- List All the Pathways and Habitat Attributes for the Action Category ---------------
    pathways_habitat_attributes_x = Habitat_Attribute_Category_Data[Habitat_Attribute_Category_Data['Action Category'].isin(action_category_x.tolist())]
    
    # ---------------- Add the Action Type to the beginning of the Data Frame --------------
    pathways_habitat_attributes_x.insert(0, 'Action Type', action_type_x )
    
    # --------------------------- Add to Main Data Frame of Action Types and Habitat Attributes ---------
    Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.append(pathways_habitat_attributes_x)
    
# -------------------- add an NA row ------------------
NA_row = pd.DataFrame(['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA'])
NA_row = NA_row.transpose()
NA_row.columns = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.columns
Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.append(NA_row)

In [630]:
# ------------ remove any white space at beginning or end of cells -----------------
Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [631]:
# ------------------------------ OUTPUT to an Excel spreadsheet  -------------------------  
Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.to_excel(output_Crosswalk_Habitat_Attributes_and_Actions, index = False)

In [632]:
# Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'] == 'Off-Channel- Side-Channels']

In [633]:
# Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].unique()

In [634]:
# Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.shape[0]

## ----------------------------------------------------------------------------------------------------------------------------
# 
#             Part 2: Generate Projects for each Reaches Based on their Habitat Scores
# 
## ----------------------------------------------------------------------------------------------------------------------------

In [635]:
# Step 1: Identify Priority Reaches (via HQ or LF) - pull up the reach (do this for all)
# Step 2: identify what habitat attributes are impaired based on a criteria (Habitat Quality or Limiting Factor?)
# Step 3: Identify Action Types/Categories linked to impacted Habitat Attributes in each reach (based on criteria in Step 1)
# Step 4: Identify Projects in the Step 1 reaches from RAs related to those Action Types/Categories


## ----------------------------------------------------------------------------------------------------------------------------
#           Open Habitat Quality Scores data
## ----------------------------------------------------------------------------------------------------------------------------

In [636]:
# -----------------------  Past for the Habitat Quality Scores --------------------
Habitat_Quality_Spring_Chinook_Pathway = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Habitat_Quality_Output_Spring_Chinook.xlsx'
Habitat_Quality_Steelhead_Pathway = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Habitat_Quality_Output_Steelhead.xlsx'

#Habitat_Quality_Spring_Chinook_Pathway = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/Habitat_Quality_Output_Spring_Chinook.xlsx'
#Habitat_Quality_Steelhead_Pathway = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/Habitat_Quality_Output_Steelhead.xlsx'


# ------------------- Read in Habitat Quality Scores ---------
Habitat_Quality_Spring_Chinook = pd.read_excel(Habitat_Quality_Spring_Chinook_Pathway)
Habitat_Quality_Steelhead = pd.read_excel(Habitat_Quality_Steelhead_Pathway)

In [637]:
# ----------------------- Replace "Error" values with NA value ----------------
Habitat_Quality_Spring_Chinook['Riparian- Structure'] = Habitat_Quality_Spring_Chinook['Riparian- Structure'].replace('Error', np.nan)
Habitat_Quality_Steelhead['Riparian- Structure'] = Habitat_Quality_Steelhead['Riparian- Structure'].replace('Error', np.nan)

## ----------------------------------------------------------------------------------------------------------------------------
#
##            Develop Function to Output Actions for Habitat Quality
#
## ----------------------------------------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------------------------------------
###            Functions to output individual impaired habitat attribute and reaches
## ----------------------------------------------------------------------------------------------------------------------------

In [638]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return(unique_list)

def output_habitat_actions_categories_for_Habitat_Quality_FUNCTION(crosswalk_habitat_attribute_x, reach_name_x, habitat_quality_row, habitat_quality_output, column_names):
    
    # ------------- identify Action Category and Action Type related to impaired habitat attribute
    pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin([crosswalk_habitat_attribute_x])]

    # ---------------------- Only get Projects developed for this specific reach ---------
    Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

    # --------------------- List Projects by Action Category for this specific impaired habitat attribute -------------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

    # -------- action category - if more than one, combine all --------
    action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
    action_category_x = unique(action_category_x)
    action_category_x = ',  '.join(action_category_x)
    # -------- action type - if more than one, combine all --------
    action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
    action_type_x = unique(action_type_x)
    action_type_x = ',  '.join(action_type_x)

    # --------------- if there is no Projects in this specific reach --------------
    if Projects_List_Specific_Reach_x.shape[0] == 0:
        output_row_x = pd.DataFrame([ habitat_quality_row['Subbasin'], habitat_quality_row['Assessment Unit'], 
                                     habitat_quality_row['ReachName'], habitat_quality_row['Habitat_Attribute'], 
                                     habitat_quality_row['Habitat_Quality_Score'], action_category_x, action_type_x,'no', 'NA'])
        output_row_x = output_row_x.transpose()
        output_row_x.columns = column_names

    # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute ----
    elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

        output_row_x = pd.DataFrame([habitat_quality_row['Subbasin'], habitat_quality_row['Assessment Unit'], 
                                     habitat_quality_row['ReachName'], habitat_quality_row['Habitat_Attribute'], 
                                     habitat_quality_row['Habitat_Quality_Score'], 
                                     action_category_x,action_type_x, 'no', 'NA'])
        output_row_x = output_row_x.transpose()
        output_row_x.columns = column_names

    # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
    else:

        # --------------------------------------------------------------------------
        #           Add project to saved output
        # --------------------------------------------------------------------------

        # ------------- action (project) descriptions ----------
        project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
        project_x = unique(project_x)
        project_x = ',  '.join(project_x)

        output_row_x = pd.DataFrame([habitat_quality_row['Subbasin'], habitat_quality_row['Assessment Unit'], 
                                     habitat_quality_row['ReachName'], habitat_quality_row['Habitat_Attribute'], 
                                     habitat_quality_row['Habitat_Quality_Score'],  
                                     action_category_x, action_type_x,'yes', project_x])
        output_row_x = output_row_x.transpose()
        output_row_x.columns = column_names
        #print(habitat_quality_row)
        #print(output_row_x)
        #print("     ")
            
    return(output_row_x)


## ----------------------------------------------------------------------------------------------------------------------------
###           Master Function for Habitat Quality
## ----------------------------------------------------------------------------------------------------------------------------

In [639]:
# Habitat_Quality_Steelhead.columns

In [640]:
# habitat_attributes

In [641]:
# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

def Generate_Project_Output_Table_for_Habitat_Quality(species):

    # ------------------------------------------------------------------------------
    #      Initiate Data Processing
    # ------------------------------------------------------------------------------
    print('   ')
    print('-------------------------- starting output for Species: ' + species + '------------------------')
    
    # ------------------------------------------------------------------------------
    #       Establish species name variable
    # ------------------------------------------------------------------------------

    if species == "Spring Chinook":

        # ------- establish species variable name -----
        variable_name = 'variable_SPCH'
        # ------- Use Spring Chinook data ----------
        criteria_data = Habitat_Quality_Spring_Chinook
        # ------------- output paths ---------
        output_location_projects_only = output_location_Spring_Chinook_Habitat_Quality_Projects_only
        output_location_all_actions = output_location_Spring_Chinook_Habitat_Quality_All_Actions
            
    elif species == "Steelhead":

        # ------- establish species variable name -----
        variable_name = 'variable_SH'
        # ------- Use Spring Chinook data ----------
        criteria_data = Habitat_Quality_Steelhead
        # ------------- output paths ---------
        output_location_projects_only = output_location_Steelhead_Habitat_Quality_Projects_only
        output_location_all_actions = output_location_Steelhead_Habitat_Quality_All_Actions
        
    else:

        
        print('Incorrectly entered species name - re-type species name')
        return
        
        
    # ------------------------------------------------------------------------------
    #      Step 1: Identify Priority Reaches (via Habitat Quality)
    # ------------------------------------------------------------------------------

    # NOTE: no actions needed for this - since Habitat Quality score data are all priority reaches
    #        based on the Habitat_Quality_Prioritization_Output.ipynb code
    
    # ------------------------------------------------------------------------------
    #      Step 2: Identify impaired habitat attributes based on Habitat Quality criteria 
    # ------------------------------------------------------------------------------

    # -------------- data frame to add reaches with specific habitat attributes that are impaired -------
    impaired_habitat_data_frame = pd.DataFrame([])
    # ------------ data frame that will include all impaired reaches -------------
    column_names = [ 'Subbasin','Assessment_Unit','ReachName', 'Habitat_Attribute', 'Habitat_Quality_Score',
                    'Action Category','Action Type','Project_present_yes_no','Project']
    habitat_quality_output = pd.DataFrame([], columns = column_names)

    for habitat_attribute_x in habitat_attributes:
        print('habitat_attribute_x')
        print(habitat_attribute_x)
        # ------------- identify all the reaches that are impaired based on the HQ score for this habitat attribute ------
        # NOTE - this is redundant - since for the HQ pathway script - it already filters for this, only necessary if pulling data that is not already filtered for the HQ score
        criteria_data_x = criteria_data[['ReachName', 'Assessment Unit', 'Subbasin',habitat_attribute_x]][criteria_data[habitat_attribute_x]<=indiv_habitat_attribute_score] 
        # ------------------------- rename column names ----------
        criteria_data_x.columns  = ['ReachName', 'Assessment Unit', 'Subbasin', 'Habitat_Quality_Score']
        # ------------------------- add habitat attribute name ----------
        criteria_data_x['Habitat_Attribute'] = habitat_attribute_x
        # ----------------------- append to data frame -------------
        impaired_habitat_data_frame = impaired_habitat_data_frame.append(criteria_data_x)
        
        # ------------ print data --------
        print('------------ Habitat Attribute: ' + habitat_attribute_x)
        print('Total number of reaches for this habitat attribute: ')
        print(criteria_data_x.shape[0])
        

    
    #print(impaired_habitat_data_frame)
    # ------------------------------------------------------------------------------
    #      Step 3:  Identify Action Types/Categories linked to impacted Habitat Attributes in each impaired reach (based on criteria in Step 1) 
    # ------------------------------------------------------------------------------

    reaches_not_present_in_projects = pd.DataFrame([])
    reaches_present_no_projects = pd.DataFrame([])
    Projects_Action_Type_for_impaired_habitat_attributes = pd.DataFrame([])
    Projects_Action_Category_for_impaired_habitat_attributes = pd.DataFrame([])
    Projects_for_impaired_habitat_attributes = pd.DataFrame([])
    for index, row in impaired_habitat_data_frame.iterrows():

                
        # ----------------- Reach Name for this Specific reach -----------
        reach_name_x = row['ReachName']

        # ---------------- Idenfity habitat attribute for this reach ---------------
        habitat_attribute_x = row['Habitat_Attribute']
        crosswalk_habitat_attribute_x = habitat_attribute_x
        
        # ----------------- Run Function to get output for each reach-impaired habitat combo, print Projects when present -----
        habitat_quality_row = row
        row_output_x = output_habitat_actions_categories_for_Habitat_Quality_FUNCTION(crosswalk_habitat_attribute_x, reach_name_x, habitat_quality_row, habitat_quality_output, column_names)
        habitat_quality_output = habitat_quality_output.append(row_output_x)  

        # -------------- create Data Frame with reach and habitat attribute -------
        reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x])
        reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

        # ------------ identify habitat attribute names connected to Action Category/Type --------
        crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

        # ------------- identify Action Category and Action Type related to impaired habitat attribute
        pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

        # ---------------------- Only get Projects developed for this specific reach ---------
        Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

        # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

        # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

        # --------------- if there is no Projects in this specific reach --------------
        if Projects_List_Specific_Reach_x.shape[0] == 0:
            reaches_not_present_in_projects = reaches_not_present_in_projects.append(reach_habitat_attribute_data_frame)

        # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----
        elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:
            reaches_present_no_projects = reaches_present_no_projects.append(reach_habitat_attribute_data_frame)

        # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
        else:
            #print('-----------------reaches with projects for impaired habitat attribute-------------')
            #print(reach_habitat_attribute_data_frame)

            # ----------------- Add the habitat attribute name to the data frame ----------------
            # ------------ For Action Type data frame ---------
            reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
            reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute']
            reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

            # --------------- For Action Category data frame -------------
            reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
            reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute']
            reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

            # --------------- save output to individual Action Category/Action Type data frames ---------
            Projects_Action_Type_for_impaired_habitat_attributes = Projects_Action_Type_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
            Projects_Action_Category_for_impaired_habitat_attributes = Projects_Action_Category_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)

            # ------------- Save output to master data frame -------------------
            Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
            Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)
            #print(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape)
            #print(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape)

    # ------------------------- Collapse duplicated projects to one project, list multiple habitat attributes ----------
    project_data_frame = pd.DataFrame(Projects_for_impaired_habitat_attributes['ProjectID'])
    duplicated_project_data_frame_x = project_data_frame.loc[project_data_frame.duplicated()].sort_values(by=['ProjectID'])

    # ------- loop through projects to collapse duplicated projects ------- 
    Projects_for_impaired_habitat_attributes_updated = pd.DataFrame([])
    unique_Project_IDs_x = Projects_for_impaired_habitat_attributes.ProjectID.unique()
    for projectID_x in unique_Project_IDs_x:

        # ---------- if a duplicated project --------
        if any(duplicated_project_data_frame_x.ProjectID.isin([projectID_x]) ): 
            Duplicated_Projects_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
            # ------- Habitat Attribute list for all the duplicated projects ----------
            Habitat_Attributes_duplicated_x = Duplicated_Projects_x.Habitat_Attribute
            Habitat_Attributes_duplicated_x = Habitat_Attributes_duplicated_x.unique()
            Habitat_Attributes_duplicated_x = str(np.stack(Habitat_Attributes_duplicated_x) )

            # ------------ Get a single row for all the duplicated projects ---------------
            row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])].iloc[0:1,]
            row_x.Habitat_Attribute = Habitat_Attributes_duplicated_x
            Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)

        # ------- if project ID is not duplicated ----------
        else:
            row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
            Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)

    # ----------------- print number of reaches -------------
    print('total number of JUST projects : ' )
    print(Projects_for_impaired_habitat_attributes_updated.shape[0] )
    print(" ")
    print(' total number of Action Categories, Action Types and Projects: ')
    print( habitat_quality_output.shape[0])
    print(" ")
    # ------------------------------------------------------------------------------
    #      Step 4: Generate output based on reaches
    # ------------------------------------------------------------------------------
    
    # --------- remove brackets from Habitat Attributes -------
    Projects_for_impaired_habitat_attributes_updated['Habitat_Attribute'] = Projects_for_impaired_habitat_attributes_updated['Habitat_Attribute'].str.strip("[]")
    
    # ------------------------------ OUTPUT to an Excel spreadsheet  -------------------------  
    Projects_for_impaired_habitat_attributes_updated.to_excel(output_location_projects_only, index = False)
    print('output location for JUST projects: ' + output_location_projects_only)
    habitat_quality_output.to_excel(output_location_all_actions, index = False)
    print('output location for Action Categories, Action Types, and Projects (when present): ' + output_location_all_actions)
    
    

    

## ----------------------------------------------------------------------------------------------------------------------------
##           Output Actions for Habitat Quality
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
Generate_Project_Output_Table_for_Habitat_Quality('Spring Chinook')
Generate_Project_Output_Table_for_Habitat_Quality('Steelhead')

   
-------------------------- starting output for Species: Spring Chinook------------------------
habitat_attribute_x
Temperature- Rearing
------------ Habitat Attribute: Temperature- Rearing
Total number of reaches for this habitat attribute: 
51
habitat_attribute_x
Flow- Summer Base Flow
------------ Habitat Attribute: Flow- Summer Base Flow
Total number of reaches for this habitat attribute: 
21
habitat_attribute_x
Riparian- Structure
------------ Habitat Attribute: Riparian- Structure
Total number of reaches for this habitat attribute: 
16
habitat_attribute_x
Riparian-Disturbance
------------ Habitat Attribute: Riparian-Disturbance
Total number of reaches for this habitat attribute: 
19
habitat_attribute_x
Riparian- Canopy Cover
------------ Habitat Attribute: Riparian- Canopy Cover
Total number of reaches for this habitat attribute: 
35
habitat_attribute_x
Riparian
------------ Habitat Attribute: Riparian
Total number of reaches for this habitat attribute: 
15
habitat_attribute_x

<ipython-input-641-981b8ec62879>:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
<ipython-input-641-981b8ec62879>:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']


total number of JUST projects : 
141
 
 total number of Action Categories, Action Types and Projects: 
304
 
output location for JUST projects: Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Prioritized_Project_List_Habitat_Quality_Output_Spring_Chinook.xlsx
output location for Action Categories, Action Types, and Projects (when present): Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Impaired_Habitat_Reaches_Actions_List_Habitat_Quality_Output_Spring_Chinook.xlsx
   
-------------------------- starting output for Species: Steelhead------------------------
habitat_attribute_x
Temperature- Rearing
------------ Habitat Attribute: Temperature- Rearing
Total number of reaches for this habitat attribute: 
67
habitat_attribute_x
Flow- Summer Base Flow
------------ Habitat Attribute: Flow- Summer Base Flow
Total number of reaches for this habitat attribute: 
20
habitat_attribute_x
Riparian- Structure
------------ Habi

## ----------------------------------------------------------------------------------------------------------------------------
#
#            Develop Function to Output Actions for Limiting Factors
#
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# ---------------- Print out Habitat Attributes for Limiting Factors -----------
Limiting_Factor_Spring_Chinook_Adult_Migration['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Spring_Chinook_Holding_and_Maturation['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Spring_Chinook_Spawning['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Spring_Chinook_Fry['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Spring_Chinook_Summer_Rearing['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Spring_Chinook_Winter_Rearing['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Spring_Chinook_Smolt['Habitat Attribute'].sort_values().unique(),

Limiting_Factor_Steelhead_Adult_Migration['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Steelhead_Holding_and_Maturation['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Steelhead_Spawning['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Steelhead_Fry['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Steelhead_Summer_Rearing['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Steelhead_Winter_Rearing['Habitat Attribute'].sort_values().unique(),
Limiting_Factor_Steelhead_Smolt['Habitat Attribute'].sort_values().unique() 


In [ ]:
#if any(Limiting_Factor_Spring_Chinook_Adult_Migration.columns == "Individual Habitat Scores") == False:
#    Limiting_Factor_Spring_Chinook_Adult_Migration["Individual Habitat Scores"] 

In [ ]:
# Limiting_Factor_Spring_Chinook_Summer_Rearing['Habitat Attribute'].unique()
# criteria_data_summer_rearing['Habitat Attribute'].unique()

In [ ]:
# -------------rename the Limiting Factor data frames for function -------
criteria_data_adult_migration = Limiting_Factor_Spring_Chinook_Adult_Migration
criteria_data_holding_and_maturation = Limiting_Factor_Spring_Chinook_Holding_and_Maturation
criteria_data_spawning = Limiting_Factor_Spring_Chinook_Spawning
criteria_data_fry = Limiting_Factor_Spring_Chinook_Fry
criteria_data_summer_rearing = Limiting_Factor_Spring_Chinook_Summer_Rearing
criteria_data_winter_rearing = Limiting_Factor_Spring_Chinook_Winter_Rearing
criteria_data_smolt = Limiting_Factor_Spring_Chinook_Smolt

## ----------------------------------------------------------------------------------------------------------------------------
###            Functions to output individual impaired limiting factor and reaches
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Limiting_Factor_Steelhead_Winter_Rearing.head()

In [ ]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return(unique_list)


def output_habitat_actions_categories_Limiting_Factor_FUNCTION(crosswalk_habitat_attribute_x, reach_name_x, life_stage_output_x,
                                                              impaired_habitat_row, limiting_factor_output, column_names):
    #print('----------impaired_habitat_row ---------:')
    #print(impaired_habitat_row)
    # ------------- identify Action Category and Action Type related to impaired habitat attribute
    pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin([crosswalk_habitat_attribute_x])]

    # ---------------------- Only get Projects developed for this specific reach ---------
    Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

    # -------- action category - if more than one, combine all --------
    action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
    action_category_x = unique(action_category_x)
    action_category_x = ',  '.join(action_category_x)
    # -------- action type - if more than one, combine all --------
    action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
    action_type_x = unique(action_type_x)
    action_type_x = ',  '.join(action_type_x)

    # --------------- if there is no Projects in this specific reach --------------
    if Projects_List_Specific_Reach_x.shape[0] == 0:
        output_row_x = pd.DataFrame([impaired_habitat_row['Subbasin'], impaired_habitat_row['Assessment Unit'], 
                                     impaired_habitat_row['ReachName'], impaired_habitat_row['Habitat Attribute'], impaired_habitat_row['Life_Stage'], 
                                     impaired_habitat_row['Habitat Attribute Score'],
                                    action_category_x, action_type_x,'no', 'NA'])
        output_row_x = output_row_x.transpose()
        output_row_x.columns =  column_names

    # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute ----
    elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

        output_row_x = pd.DataFrame([ impaired_habitat_row['Subbasin'], impaired_habitat_row['Assessment Unit'], 
                                     impaired_habitat_row['ReachName'], impaired_habitat_row['Habitat Attribute'], impaired_habitat_row['Life_Stage'], 
                                     impaired_habitat_row['Habitat Attribute Score'],  
                                     action_category_x,action_type_x, 'no', 'NA'])
        output_row_x = output_row_x.transpose()
        output_row_x.columns =  column_names

    # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
    else:

        # --------------------------------------------------------------------------
        #           Add project to saved output
        # --------------------------------------------------------------------------

        # ------------- action (project) descriptions ----------
        project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
        project_x = unique(project_x)
        project_x = ',  '.join(project_x)

        output_row_x = pd.DataFrame([ impaired_habitat_row['Subbasin'], impaired_habitat_row['Assessment Unit'], 
                                     impaired_habitat_row['ReachName'], impaired_habitat_row['Habitat Attribute'], impaired_habitat_row['Life_Stage'], 
                                     impaired_habitat_row['Habitat Attribute Score'],  
                                     action_category_x, action_type_x,'yes', project_x])
        output_row_x = output_row_x.transpose()
        output_row_x.columns =  column_names
            
    return(output_row_x)


## ----------------------------------------------------------------------------------------------------------------------------
###            Master Function for Limiting Factors
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Limiting_Factor_Spring_Chinook_Holding_and_Maturation[Limiting_Factor_Spring_Chinook_Holding_and_Maturation['Habitat Attribute'] == 'Pools- Deep Pools']

In [ ]:
# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

def Generate_Project_Output_Table_for_Limiting_Factors(species, indiv_habitat_attribute_score):

    # ------------------------------------------------------------------------------
    #      Initiate Data Processing
    # ------------------------------------------------------------------------------

    print("       ")
    print('======================= starting output for Species: ' + species + '=========================')

    
    # ------------------------------------------------------------------------------
    #       Establish species name variable
    # ------------------------------------------------------------------------------

    if species == "Spring Chinook":

        # ------- establish species variable name -----
        variable_name = 'variable_SPCH'
        # ------- Use Spring Chinook data ----------
        criteria_data_adult_migration = Limiting_Factor_Spring_Chinook_Adult_Migration
        criteria_data_holding_and_maturation = Limiting_Factor_Spring_Chinook_Holding_and_Maturation
        criteria_data_spawning = Limiting_Factor_Spring_Chinook_Spawning
        criteria_data_fry = Limiting_Factor_Spring_Chinook_Fry
        criteria_data_summer_rearing = Limiting_Factor_Spring_Chinook_Summer_Rearing
        criteria_data_winter_rearing = Limiting_Factor_Spring_Chinook_Winter_Rearing
        criteria_data_smolt = Limiting_Factor_Spring_Chinook_Smolt
        # ------------- output paths ---------
        output_location_all_projects = output_location_Spring_Chinook_Limiting_Factors_All_Projects
        output_location_all_actions = output_location_Spring_Chinook_Limiting_Factors_All_Actions
        
    elif species == "Steelhead":

        # ------- establish species variable name -----
        variable_name = 'variable_SH'
        # ------- Use Spring Chinook data ----------
        criteria_data_adult_migration = Limiting_Factor_Steelhead_Adult_Migration
        criteria_data_holding_and_maturation = Limiting_Factor_Steelhead_Holding_and_Maturation
        criteria_data_spawning = Limiting_Factor_Steelhead_Spawning
        criteria_data_fry = Limiting_Factor_Steelhead_Fry
        criteria_data_summer_rearing = Limiting_Factor_Steelhead_Summer_Rearing
        criteria_data_winter_rearing = Limiting_Factor_Steelhead_Winter_Rearing
        criteria_data_smolt = Limiting_Factor_Steelhead_Smolt
        # ------------- output paths ---------
        output_location_all_projects = output_location_Steelhead_Limiting_Factors_All_Projects
        output_location_all_actions = output_location_Steelhead_Limiting_Factors_All_Actions
        
    else:

        print('Incorrectly entered species name - re-type species name')
        return
        
        
    # ------------------------------------------------------------------------------
    #      Step 1: Identify Priority Reaches (via Limiting Factors)
    # ------------------------------------------------------------------------------

    # NOTE: no actions needed for this - since Habitat Quality score data are all priority reaches
    #        based on the Habitat_Quality_Prioritization_Output.ipynb code
    
    # ------------------------------------------------------------------------------
    #      Step 2: Identify impaired habitat attributes based on Limiting Factors criteria 
    # ------------------------------------------------------------------------------

    # -------------- data frame to add reaches with specific habitat attributes that are impaired -------
    impaired_habitat_data_frame = pd.DataFrame([])
    # ------------ data frame that will include all impaired reaches -------------
    column_names = [ 'Subbasin','Assessment_Unit','ReachName', 'Habitat_Attribute','Life_Stage', 
                    'Habitat_Attribute_Score','Action Category','Action Type','Project_present_yes_no','Project']
    limiting_factor_output = pd.DataFrame([], columns = column_names)
    print_life_stage = ['adult_migration', 'holding_and_maturation', 'spawning', 'fry', 'summer_rearing', 'winter_rearing',  'smolt']
    print_i = 0
    for criteria_data in (criteria_data_adult_migration, criteria_data_holding_and_maturation, criteria_data_spawning, criteria_data_fry, criteria_data_summer_rearing, criteria_data_winter_rearing,  criteria_data_smolt):
        print("   ")
        print(' ------------  Starting Life Stage:   ' + print_life_stage[print_i] + ' ------------------ ')
        print_i = print_i + 1
        
        # -------- go to next if not present ---
        print(" rows in criteria data")
        print(criteria_data.shape[0])
        if criteria_data.shape[0] == 0:
            print(' no data in limiting factor output for this life stage')
            continue
        
        # ---------------- generate unique habitat attributes for this life stage -----------
        habitat_attributes = criteria_data['Habitat Attribute'].sort_values().unique()

        # ----------- loop through habitat attributes -------------
        for habitat_attribute_x in habitat_attributes:
            
            print('---------------- habitat attribute x  -----------------------')
            print(habitat_attribute_x)

            # -------- if habitat attribute is in crosswalk list -------
            boolean_output = Habitat_Attribute_Names_Crosswalk_df == habitat_attribute_x
            boolean_T_F = boolean_output.any(axis = 'columns')
            if boolean_T_F.any():

                # ------------ generate criteria data just with habitat_attribute_x -----------
                criteria_data_habitat_attributre_x = criteria_data.loc[criteria_data['Habitat Attribute'].isin([habitat_attribute_x])]
                #print('-----------criteria_data_habitat_attributre_x -----------:')
                #print(criteria_data_habitat_attributre_x)
                # ------------- identify all the reaches that are impaired based on the habitat_attribute_x attribute ------
                criteria_data_x = criteria_data_habitat_attributre_x[['ReachName', 'Assessment Unit', 'Subbasin','Habitat Attribute', 'Individual Habitat Attribute Score']][criteria_data_habitat_attributre_x['Individual Habitat Attribute Score']<=indiv_habitat_attribute_score]
                # ------------------------- rename column names ----------
                criteria_data_x.columns  = ['ReachName', 'Assessment Unit', 'Subbasin', 'Habitat Attribute','Habitat Attribute Score']
                # ------------------------- add habitat attribute name ----------
                criteria_data_x['Habitat_Attribute'] = habitat_attribute_x
                # ------------------------- add habitat attribute name ----------
                criteria_data_x['Life_Stage'] = criteria_data['Life_Stage'][0]
                # ----------------------- append to data frame -------------
                impaired_habitat_data_frame = impaired_habitat_data_frame.append(criteria_data_x)
                
                # ------------ print data --------
                print('Habitat Attribute: ' + habitat_attribute_x)
                print('Total number of reaches for this life stage and habitat attribute: ')
                print(criteria_data_x.shape[0])

        
    # ------------------------------------------------------------------------------
    #      Step 3:  Identify Action Types/Categories linked to impacted Habitat Attributes in each impaired reach (based on criteria in Step 1) 
    # ------------------------------------------------------------------------------
    print(" STARTING STEP 3")
    print("impaired_habitat_data_frame shape")
    print(impaired_habitat_data_frame.shape)
    reaches_not_present_in_projects = pd.DataFrame([])
    reaches_present_no_projects = pd.DataFrame([])
    Projects_Action_Type_for_impaired_habitat_attributes = pd.DataFrame([])
    Projects_Action_Category_for_impaired_habitat_attributes = pd.DataFrame([])
    Projects_for_impaired_habitat_attributes = pd.DataFrame([])

    for index, row in impaired_habitat_data_frame.iterrows():
        print('row')
        print(row)

        # ----------------- Reach Name for this Specific reach -----------
        reach_name_x = row['ReachName']

        # ---------------- Idenfity habitat attribute for this reach ---------------
        habitat_attribute_x = row['Habitat_Attribute']
        crosswalk_habitat_attribute_x = habitat_attribute_x
        
        # ---------------- Idenfity habitat attribute for this reach ---------------
        life_stage_x = row['Life_Stage']
        
        # ---------------- Idenfity habitat attribute for this reach ---------------
        habitat_attribute_score_x = row['Habitat Attribute Score']
        #print('-----------------------row')
        #print(row)
        #print('habitat_attribute_score_x --------------------')
        #print(habitat_attribute_score_x)
        
        # ----------------- Run Function to get output for each reach-impaired habitat combo, print Projects when present -----
        row_output_x = output_habitat_actions_categories_Limiting_Factor_FUNCTION(crosswalk_habitat_attribute_x, reach_name_x, life_stage_x, row, limiting_factor_output, column_names)
        limiting_factor_output = limiting_factor_output.append(row_output_x)  


        # -------------- create Data Frame with reach and habitat attribute -------
        reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x, life_stage_x, habitat_attribute_score_x])
        reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

        # ------------ identify habitat attribute names connected to Action Category/Type --------
        crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

        # ------------- identify Action Category and Action Type related to impaired habitat attribute
        pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

        # ---------------------- Only get Projects developed for this specific reach ---------
        Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

        # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

        # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

        #print(reach_name_x)
        #print(Projects_List_Specific_Reach_x.shape)
        #print(Projects_List_Specific_Reach_x_Habitat_Attribute.shape)

        # --------------- if there is no Projects in this specific reach --------------
        if Projects_List_Specific_Reach_x.shape[0] == 0:
            reaches_not_present_in_projects = reaches_not_present_in_projects.append(reach_habitat_attribute_data_frame)

        # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----
        elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:
            reaches_present_no_projects = reaches_present_no_projects.append(reach_habitat_attribute_data_frame)

        # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
        else:
            #print('-----------------reaches with projects for impaired habitat attribute-------------')
            #print(reach_habitat_attribute_data_frame)

            # ----------------- Add the habitat attribute name to the data frame ----------------
            
            # --------------------------------------------------------------------------
            #                      For Action Type data frame 
            # --------------------------------------------------------------------------

            reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
            reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute','Life_Stage','Limiting_Factor_Score']
            reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index
            
            # ------------------- add Habitat Attribute name -----------
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
            # ----------------------- add life stage --------------
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x
            # ----------------------- add life stage --------------
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute_Score'] = habitat_attribute_score_x
        
            # --------------------------------------------------------------------------
            #                      For Action Category data frame 
            # --------------------------------------------------------------------------
            reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
            reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute', 'Life_Stage','Habitat_Attribute_Score']
            reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index
            
            # ------------------- add Habitat Attribute name -----------
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
            
            # ----------------------- add life stage --------------
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x
        
            # ----------------------- add limiting factor --------------
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute_Score'] = habitat_attribute_score_x
        
            # --------------- save output to individual Action Category/Action Type data frames ---------
            Projects_Action_Type_for_impaired_habitat_attributes = Projects_Action_Type_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
            Projects_Action_Category_for_impaired_habitat_attributes = Projects_Action_Category_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)

            # --------------------------------------------------------------------------
            #           Save output for Action Category and Action Types
            # --------------------------------------------------------------------------

            Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
            Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)
            #print(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape)
            #print(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape)
            print(Projects_for_impaired_habitat_attributes.shape)
            

    
    # ------------------------- Collapse duplicated projects to one project, list multiple habitat attributes ----------
    #Projects_for_impaired_habitat_attributes['Habitat_Attribute'] = Projects_for_impaired_habitat_attributes['Habitat_Attribute'].strip("[]")
    print(Projects_for_impaired_habitat_attributes.shape)
    project_data_frame = pd.DataFrame(Projects_for_impaired_habitat_attributes['ProjectID'])
    duplicated_project_data_frame_x = project_data_frame.loc[project_data_frame.duplicated()].sort_values(by=['ProjectID'])
    
    # ------- loop through projects to collapse duplicated projects ------- 
    Projects_for_impaired_habitat_attributes_updated = pd.DataFrame([])
    unique_Project_IDs_x = Projects_for_impaired_habitat_attributes.ProjectID.unique()
    for projectID_x in unique_Project_IDs_x:

        # ---------- if a duplicated project --------
        if any(duplicated_project_data_frame_x.ProjectID.isin([projectID_x]) ): 
            Duplicated_Projects_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
            # ------- Habitat Attribute list for all the duplicated projects ----------
            Habitat_Attributes_duplicated_x = Duplicated_Projects_x.Habitat_Attribute
            Habitat_Attributes_duplicated_x = Habitat_Attributes_duplicated_x.unique()
            Habitat_Attributes_duplicated_x = str(np.stack(Habitat_Attributes_duplicated_x) )

            # ------------ Get a single row for all the duplicated projects ---------------
            #print('---------------- Projects_for_impaired_habitat_attributes  Life Stage')
            #print(Projects_for_impaired_habitat_attributes['Life_Stage'])
            row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])].iloc[0:1,]
            row_x.Habitat_Attribute = Habitat_Attributes_duplicated_x
            Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)
            #print(' ------------------ row_x - duplicated project --------------')
            #print(row_x['Life_Stage'])

        # ------- if project ID is not duplicated ----------
        else:
            row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
            Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)
            #print(' ------------------ row_x - not duplicated--------------')
            #print(row_x['Life_Stage'])
         
    print('total number of JUST projects pre-confinement filter : ' )
    print(Projects_for_impaired_habitat_attributes_updated.shape[0] )
    limiting_factor_output_SHAPE_PRINT = limiting_factor_output.shape
    
    # ------------------------------------------------------------------------------
    #      Step 3B: Filter out actions based on confinment (i.e. don't pull actions requiring unconfined areas)
    # ------------------------------------------------------------------------------
              
    for action_x in Reach_Confinement_Limiting_Factors:
        
        # ------ Identify Reaches with Action ---------
        projects_with_action = Projects_for_impaired_habitat_attributes_updated.loc[ Projects_for_impaired_habitat_attributes_updated['Action_Category']==action_x ]
        limiting_factor_output_with_action =  limiting_factor_output.loc[ limiting_factor_output['Action Category'] == action_x]

        # --------Identify which of those Reaches is unconfined -----------
        Confinement_Criteria = confinement_scores.loc[confinement_scores[Reach_Confinement_vars['variable'][0]].isin(Reach_Confinement_Limiting_Factors[action_x])]
        Confinement_Criteria_Reaches = Confinement_Criteria.ReachName
        
        # ------- Identify Reaches with Confinment Criteria -------------
        projects_with_action = projects_with_action.loc[projects_with_action['Reach_UCSRB'].isin(Confinement_Criteria_Reaches)] 
        limiting_factor_output_with_action = limiting_factor_output_with_action.loc[limiting_factor_output_with_action['ReachName'].isin(Confinement_Criteria_Reaches)] 
        
        # ------------- Pull out all acions related to actoin (then put back in actions that meet confinement criteria) ------
        Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.loc[ Projects_for_impaired_habitat_attributes_updated['Action_Category'] != action_x ]
        limiting_factor_output =  limiting_factor_output.loc[ limiting_factor_output['Action Category'] != action_x]
        # ---------- Add actions that meet confinement criteria ------
        Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(projects_with_action)
        limiting_factor_output = limiting_factor_output.append(   limiting_factor_output_with_action)
        
    # -----------------------------------------------------------------------------
    #      Step 4: Generate output based on reaches
    # ------------------------------------------------------------------------------
    
    # --------- remove brackets from Habitat Attributes -------
    Projects_for_impaired_habitat_attributes_updated['Habitat_Attribute'] = Projects_for_impaired_habitat_attributes_updated['Habitat_Attribute'].str.strip("[]")
    
    print('total number of JUST projects post-confinement filter : ' )
    print(Projects_for_impaired_habitat_attributes_updated.shape[0] )
    print(" ")
    print(' total number of Action Categories, Action Types and Projects pre-confinement filter: ')
    print( limiting_factor_output_SHAPE_PRINT[0])
    print(' total number of Action Categories, Action Types and Projects post-confinement filter: ')
    print( limiting_factor_output.shape[0])

    
    # ------------------------------ OUTPUT to an Excel spreadsheet  -------------------------  
    Projects_for_impaired_habitat_attributes_updated.to_excel(output_location_all_projects, index = False)
    print('output location for JUST projects: ' + output_location_all_projects)
    limiting_factor_output.to_excel(output_location_all_actions, index = False)
    print('output location for Action Categories, Action Types, and Projects (when present):  ' + output_location_all_actions)

    

In [ ]:
# Reach_Confinement_Limiting_Factors
# Habitat_Attribute_Names_Crosswalk_df

## ----------------------------------------------------------------------------------------------------------------------------
##           Output Actions for Limiting Factors
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
Generate_Project_Output_Table_for_Limiting_Factors('Spring Chinook', indiv_habitat_attribute_score)
Generate_Project_Output_Table_for_Limiting_Factors('Steelhead', indiv_habitat_attribute_score)

## ----------------------------------------------------------------------------------------------------------------------------
# 
#             Part 3: Integrate Habitat Quality and Limiting Factor Output so no redundant projects/actoin categories 
# 
## ----------------------------------------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------------------------------------
###             Read in Output (that was generated by this script)
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# ----------------------------- Habitat Quality Output ---------------------
Habitat_Quality_Spring_Chinook_All_Actions = pd.read_excel(output_location_Spring_Chinook_Habitat_Quality_All_Actions)
Habitat_Quality_Steelhead_All_Actions = pd.read_excel(output_location_Steelhead_Habitat_Quality_All_Actions)

# ------------------------------ Limiting Factor Output ------------------
Limiting_Factors_Spring_Chinook_All_Actions = pd.read_excel(output_location_Spring_Chinook_Limiting_Factors_All_Actions)
Limiting_Factors_Steelhead_All_Actions = pd.read_excel(output_location_Steelhead_Limiting_Factors_All_Actions)


In [ ]:
# output_location_Spring_Chinook_Habitat_Quality_All_Actions

In [ ]:
# Habitat_Quality_Spring_Chinook_All_Actions[Habitat_Quality_Spring_Chinook_All_Actions.Habitat_Attribute== 'Pool Quantity& Quality' ]

## ----------------------------------------------------------------------------------------------------------------------------
###             Combine 
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
Habitat_Quality_Spring_Chinook_All_Actions.ReachName.unique().shape

In [ ]:
Habitat_Quality_Steelhead_All_Actions.ReachName.unique().shape

In [ ]:
Limiting_Factors_Spring_Chinook_All_Actions.ReachName.unique().shape

In [ ]:
Limiting_Factors_Steelhead_All_Actions.ReachName.unique().shape

In [ ]:
# ------- since HQ pathway not explicitly linked to life stage --------
Habitat_Quality_Spring_Chinook_All_Actions['Life_Stage'] = "from_HQ_pathway"
Habitat_Quality_Steelhead_All_Actions['Life_Stage'] = "from_HQ_pathway"

In [ ]:
# -------------------- Get Unique Reaches ------------
reaches_all_x = Habitat_Quality_Spring_Chinook_All_Actions[['Subbasin', 'Assessment_Unit', 'ReachName', 'Habitat_Attribute', 'Life_Stage', 'Action Category', 'Action Type','Project_present_yes_no', 'Project']]
reaches_all_x['Pathway'] = "Habitat_Quality_Spring_Chinook"

# ------ HQ-Steelhead ------
datax = Habitat_Quality_Steelhead_All_Actions[['Subbasin', 'Assessment_Unit', 'ReachName', 'Habitat_Attribute','Life_Stage', 'Action Category', 'Action Type','Project_present_yes_no', 'Project']]
datax['Pathway'] = "Habitat_Quality_Steelhead"
reaches_all_x = reaches_all_x.append(datax)

# ------ LF-Spring Chinook ------
datax = Limiting_Factors_Spring_Chinook_All_Actions[['Subbasin', 'Assessment_Unit', 'ReachName', 'Habitat_Attribute','Life_Stage', 'Action Category', 'Action Type','Project_present_yes_no', 'Project']]
datax['Pathway'] = "Limiting_Factors_Spring_Chinook"
reaches_all_x = reaches_all_x.append(datax)

# ------ LF-Steelhead ------
datax = Limiting_Factors_Steelhead_All_Actions[['Subbasin', 'Assessment_Unit', 'ReachName', 'Habitat_Attribute','Life_Stage', 'Action Category', 'Action Type','Project_present_yes_no', 'Project']]
datax['Pathway'] = "Limiting_Factors_Steelhead"
reaches_all_x = reaches_all_x.append(datax)

In [ ]:
# reaches_all_x .Life_Stage.unique()

# -----------------------------------------------------------------------------------
##  Generate Output with Each individual Reach and Habitat Attribute has own row 
##          (multiple Action Categories for a row)
# -----------------------------------------------------------------------------------


In [ ]:

# ------------------------ Start with Habitat_Quality_Steelhead_All_Actions, see if there are othe reaches ------------
reaches_unique_x = reaches_all_x.ReachName.sort_values().unique()

HQ_LF_combined_Actions = pd.DataFrame([])
for reach_x in reaches_unique_x :
    
    # ------ identify all the data in this reach ---------
    reach_data_all_x = reaches_all_x.loc[reaches_all_x.ReachName.isin([reach_x])]
    
    # --------generate unique action categories for actions in reach -------
    actions_unique_x = reach_data_all_x['Action Category'].unique()
    
    # -------------- loop through action categories to get which pathways generated each action ----
    for action_category_x in actions_unique_x:
        
        # -------------------- get data for just this individual action category ----------
        action_category_data_all_x = reach_data_all_x.loc[reach_data_all_x['Action Category'].isin([action_category_x])]
        
        # ------------------------- identify which of the pathways included this action category ---------
        if any(action_category_data_all_x.Pathway ==  'Habitat_Quality_Spring_Chinook'):
            pathway_HQ_Spring_Chinook = 1
        else:
            pathway_HQ_Spring_Chinook = 0
            
        if any(action_category_data_all_x.Pathway ==  'Habitat_Quality_Steelhead'):
            pathway_HQ_Steelhead = 1
        else:
            pathway_HQ_Steelhead = 0
            
        if any(action_category_data_all_x.Pathway ==  'Limiting_Factors_Spring_Chinook'):
            pathway_LF_Spring_Chinook = 1
        else:
            pathway_LF_Spring_Chinook = 0
            
        if any(action_category_data_all_x.Pathway ==  'Limiting_Factors_Steelhead'):
            pathway_LF_Steelhead = 1
        else:
            pathway_LF_Steelhead = 0
            
        # ---------------- generate new row of data -----------
        row_x = pd.DataFrame(action_category_data_all_x[['Subbasin', 'Assessment_Unit', 'ReachName', 'Habitat_Attribute','Life_Stage',
       'Action Category', 'Action Type', 'Project_present_yes_no', 'Project']])
        row_x['Pathway_HQ_Spring_Chinook'] = pathway_HQ_Spring_Chinook
        row_x['Pathway_HQ_Steelhead'] = pathway_HQ_Steelhead
        row_x['Pathway_LF_Spring_Chinook'] = pathway_LF_Spring_Chinook
        row_x['Pathway_LF_Steelhead'] = pathway_LF_Steelhead
        
        HQ_LF_combined_Actions = HQ_LF_combined_Actions.append(row_x)

# ------------------------------------------------------------------------
## Output the data
# ------------------------------------------------------------------------

In [ ]:
output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/'
# output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/'

file_output = output_location + 'Actions_Habitat_Qual_and_Limit_Factors_Pathways.xlsx'
HQ_LF_combined_Actions.to_excel(file_output, index = False)

# -----------------------------------------------------------------------------------
##  Generate Output with Each individual Reach and Action Category has own row 
##          
# -----------------------------------------------------------------------------------

In [ ]:

# ------------------------ Start with Habitat_Quality_Steelhead_All_Actions, see if there are othe reaches ------------
reaches_unique_x = reaches_all_x.ReachName.sort_values().unique()

HQ_LF_combined_Actions_Unique = pd.DataFrame([])
for reach_x in reaches_unique_x :
    
    # ------ identify all the data in this reach ---------
    reach_data_all_x = reaches_all_x.loc[reaches_all_x.ReachName.isin([reach_x])]
    
    # --------generate data frame with row for each action category -------
    action_categories_unique_x = pd.DataFrame([])
    for index, row in reach_data_all_x.iterrows():
        actions_x = str(row['Action Category']).split(',') 
        for action_x_B in actions_x:
            rowx2 = row[['Subbasin', 'Assessment_Unit', 'ReachName', 'Habitat_Attribute', 'Life_Stage',
            'Action Category', 'Action Type', 'Project_present_yes_no', 'Project', 'Pathway']]
            action_x_B = pd.DataFrame([action_x_B])
            action_x_B.index = ['Habitat_Attribute_1']
            rowx2 = rowx2.append(action_x_B)
            rowx2 = rowx2.transpose()
            action_categories_unique_x = action_categories_unique_x.append(rowx2)
    #action_categories_unique_x = action_categories_unique_x.append(actions_x)

    # ---------- get unique list of the individual action categories -------
    habitat_actions_unique_x = action_categories_unique_x['Habitat_Attribute_1'].unique()

    # ------------
    # ----------- Loop through individual action categories ----------
    for habitat_act_x in habitat_actions_unique_x:

        # ------------- get reaches with this habitat actoin -------------
        reach_data_habitat_action_x = action_categories_unique_x.loc[action_categories_unique_x.Habitat_Attribute_1.isin([habitat_act_x])]

        # --------------------- Identify if multiple substrates or not -------------
        habitat_attributes_x = reach_data_habitat_action_x.Habitat_Attribute.tolist()
        if len(habitat_attributes_x) > 1:
            habitat_attributes_x = ', '.join(habitat_attributes_x)
        
        # --------------- copy life stage ----
        life_stage_x = reach_data_habitat_action_x.Life_Stage
        
        # ------------------------- identify which of the pathways included this action category ---------
        if any(reach_data_habitat_action_x.Pathway ==  'Habitat_Quality_Spring_Chinook'):
            pathway_HQ_Spring_Chinook = 1
        else:
            pathway_HQ_Spring_Chinook = 0
            
        if any(reach_data_habitat_action_x.Pathway ==  'Habitat_Quality_Steelhead'):
            pathway_HQ_Steelhead = 1
        else:
            pathway_HQ_Steelhead = 0
            
        if any(reach_data_habitat_action_x.Pathway ==  'Limiting_Factors_Spring_Chinook'):
            pathway_LF_Spring_Chinook = 1
        else:
            pathway_LF_Spring_Chinook = 0
            
        if any(reach_data_habitat_action_x.Pathway ==  'Limiting_Factors_Steelhead'):
            pathway_LF_Steelhead = 1
        else:
            pathway_LF_Steelhead = 0
            
        # ---------------- generate new row of data -----------
        row_x = pd.DataFrame(action_categories_unique_x[['Subbasin', 'Assessment_Unit', 'ReachName']].iloc[0]).transpose()
        row_x['Action Category'] = habitat_act_x
        row_x['Habitat_Attributes'] = habitat_attributes_x
        life_stage_x = life_stage_x.unique()
        if len(life_stage_x) > 1:
            life_stage_x = ' '.join(life_stage_x)
            #print('more than one life stage')
            #print(life_stage_x)
        else:
            life_stage_x = life_stage_x[0]
            #print('one life stage')
            #print(life_stage_x)
            
        row_x['Life_Stage'] = life_stage_x
        
        row_x['Pathway_HQ_Spring_Chinook'] = pathway_HQ_Spring_Chinook
        row_x['Pathway_HQ_Steelhead'] = pathway_HQ_Steelhead
        row_x['Pathway_LF_Spring_Chinook'] = pathway_LF_Spring_Chinook
        row_x['Pathway_LF_Steelhead'] = pathway_LF_Steelhead
        
        HQ_LF_combined_Actions_Unique = HQ_LF_combined_Actions_Unique.append(row_x)

In [ ]:
# reaches_all_x.Life_Stage.unique()
# HQ_LF_combined_Actions_Unique.head()

In [ ]:
# ------------ remove any white space at beginning or end of attributes -----------------
HQ_LF_combined_Actions_Unique = HQ_LF_combined_Actions_Unique.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# ------------------------------------------------------------------------
## Output the data
# ------------------------------------------------------------------------

In [ ]:
output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/'
# output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/DRAFT_Step2_Results/Habitat_Attribute_Score_1_only/'

file_output = output_location + 'Actions_Habitat_Qual_and_Limit_Factors_Pathways_Indiv_Action_Categories_Updated.xlsx'
HQ_LF_combined_Actions_Unique.to_excel(file_output, index = False)

In [ ]:
# HQ_LF_combined_Actions_Unique.Habitat_Attributes.unique()

# --------------------------------------------------------------
# #
#        Output Limiting Factors and Actions (for Sept 15 PWG Meeting)
# #
# --------------------------------------------------------------


# --------------------------------------------------------------
#       Function for Projects
# --------------------------------------------------------------

In [418]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return(unique_list)


def output_habitat_actions_categories_FUNCTION(crosswalk_habitat_attribute_x, reach_name_x, life_stage_output_x):
    
    # ------------- identify Action Category and Action Type related to impaired habitat attribute
    pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

    # ---------------------- Only get Projects developed for this specific reach ---------
    Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

    # -------- action category - if more than one, combine all --------
    action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
    action_category_x = unique(action_category_x)
    action_category_x = ',  '.join(action_category_x)
    # -------- action type - if more than one, combine all --------
    action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
    action_type_x = unique(action_type_x)
    action_type_x = ',  '.join(action_type_x)

    # --------------- if there is no Projects in this specific reach --------------
    if Projects_List_Specific_Reach_x.shape[0] == 0:
        output_row_x = pd.DataFrame([life_stage_output_x['Species'], life_stage_output_x['Life_Stage'], life_stage_output_x['Subbasin'],
                                     life_stage_output_x['Assessment_Unit'], life_stage_output_x['ReachName'], life_stage_output_x['Habitat_Attribute'], 
                                     life_stage_output_x['Habitat_Attribute_Score'], action_category_x, action_type_x,'no', 'NA'])
        output_row_x = output_row_x.transpose()
        output_row_x.columns = [[life_stage_output.columns[0],life_stage_output.columns[1],life_stage_output.columns[2],
                                 life_stage_output.columns[3],life_stage_output.columns[4],life_stage_output.columns[5],
                                  life_stage_output.columns[6],
                                 'Action Category','Action Type','Project_present_yes_no','Project']]

    # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute ----
    elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

        output_row_x = pd.DataFrame([life_stage_output_x['Species'], life_stage_output_x['Life_Stage'], life_stage_output_x['Subbasin'],
                                     life_stage_output_x['Assessment_Unit'], life_stage_output_x['ReachName'], life_stage_output_x['Habitat_Attribute'], 
                                     life_stage_output_x['Habitat_Attribute_Score'],  
                                     action_category_x,action_type_x, 'no', 'NA'])
        output_row_x = output_row_x.transpose()
        output_row_x.columns = [[life_stage_output.columns[0],life_stage_output.columns[1],life_stage_output.columns[2],
                                 life_stage_output.columns[3],life_stage_output.columns[4],life_stage_output.columns[5],
                                 life_stage_output.columns[6],
                                 'Action Category','Action Type','Project_present_yes_no','Project']]

    # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
    else:

        # --------------------------------------------------------------------------
        #           Add project to saved output
        # --------------------------------------------------------------------------

        # ------------- action (project) descriptions ----------
        project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
        project_x = unique(project_x)
        project_x = ',  '.join(project_x)

        output_row_x = pd.DataFrame([life_stage_output_x['Species'], life_stage_output_x['Life_Stage'], life_stage_output_x['Subbasin'],
                                     life_stage_output_x['Assessment_Unit'], life_stage_output_x['ReachName'], life_stage_output_x['Habitat_Attribute'],
                                    life_stage_output_x['Limiting_Factor_Score'],  
                                     action_category_x, action_type_x,'yes', project_x])
        output_row_x = output_row_x.transpose()
        output_row_x.columns = [[life_stage_output.columns[0],life_stage_output.columns[1],life_stage_output.columns[2],
                                 life_stage_output.columns[3],life_stage_output.columns[4],life_stage_output.columns[5],
                                 life_stage_output.columns[6],
                                 'Action Category','Action Type','Project_present_yes_no','Project']]
            
    return(output_row_x)


# --------------------------------------------------------------
#       Spring Chinook
# --------------------------------------------------------------

In [427]:
# --------------------------------------------------------------
#        Initial Variables 
# --------------------------------------------------------------
species_x ='Spring Chinook'

# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

# -------------- habitat attributes from limiting factor output -----------
habitat_attributes = ['Temperature',
'Flow- Summer Base Flow',  'Substrate', 'Cover- Wood',
'Pools- All Pools', 'Off-Channel- Floodplain',
'Off-Channel- Side-Channels', 'Channel Stability',
'Bank Stability', 'Riparian']

# ---------------- data frame to combine with projects --------------
projects_all_reaches_Spring_Chinook_df = pd.DataFrame([])

# -------------- data frame to combine within life stage data --------
life_stage_Spring_Chinook_output = pd.DataFrame([])

# --------------------------------------------------------------------------
#                  Get Basic info for the reach
# --------------------------------------------------------------------------

life_stage_output_x = Limiting_Factor_Spring_Chinook_ALL_Reaches[['Species', 'Life_Stage', 'Subbasin', 'Assessment Unit','ReachName', 'Habitat Attribute', 'Habitat Attribute Score']]
life_stage_output_x.columns = ['Species', 'Life_Stage','Subbasin', 'Assessment_Unit','ReachName','Habitat_Attribute', 'Limiting_Factor_Score']
life_stage_Spring_Chinook_output = life_stage_Spring_Chinook_output.append(life_stage_output_x)

# --------------------------------------------------------------------------
#               Input for Projects Processing
# --------------------------------------------------------------------------

# ----------------- Reach Name for this Specific reach -----------
reach_name_x = Limiting_Factor_Spring_Chinook_ALL_Reaches['ReachName']

# ---------------- Idenfity habitat attribute for this reach ---------------
habitat_attribute_x = Limiting_Factor_Spring_Chinook_ALL_Reaches['Habitat Attribute']

# ---------------- Idenfity habitat attribute for this reach ---------------
life_stage_x = Limiting_Factor_Spring_Chinook_ALL_Reaches['Life_Stage']

# -------------- create Data Frame with reach and habitat attribute -------
reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x, habitat_attribute_x, life_stage_x])
reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

# ------------ identify habitat attribute names connected to Action Category/Type --------
crosswalk_habitat_attribute_output_x = pd.DataFrame([])
for habitat_attribute_x in reach_habitat_attribute_data_frame['Habitat Attribute']: 
    crosswalk_habitat_attribute_output_x = crosswalk_habitat_attribute_output_x.append(Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]) 
crosswalk_habitat_attribute_output_x.columns = ['Habitat_Attribute']

# ---------------------------------------------------------------------------
#      identify action categories, types, and projects (when project is linked via crosswalk to habitat attributes)
# ---------------------------------------------------------------------------

for i in range(0,reach_habitat_attribute_data_frame.shape[0]):
    crosswalk_x = crosswalk_habitat_attribute_output_x.iloc[i]
    reach_name_x  = reach_habitat_attribute_data_frame['ReachName'].iloc[i]
    life_stage_x = life_stage_Spring_Chinook_output.iloc[i]
    output_x = output_habitat_actions_categories_FUNCTION(crosswalk_x, reach_name_x, life_stage_x)
    projects_all_reaches_Spring_Chinook_df = projects_all_reaches_Spring_Chinook_df.append(output_x)


In [181]:
reach_habitat_attribute_data_frame.head()

NameError: name 'reach_habitat_attribute_data_frame' is not defined

### ------------------------------------------------------------------------------------------------------------------
###          Output Data
### ------------------------------------------------------------------------------------------------------------------

In [360]:
output_location_COMPLETE_Spring_Chinook_Limiting_Factors

'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/COMPLETE_Project_List_Limiting_Factors_Output_Spring_Chinook_ALL_reaches.xlsx'

In [428]:
projects_all_reaches_Spring_Chinook_df.to_excel(output_location_COMPLETE_Spring_Chinook_Limiting_Factors, index=False)

# --------------------------------------------------------------
#        Steelhead
# --------------------------------------------------------------

In [429]:
# --------------------------------------------------------------
#        Initial Variables 
# --------------------------------------------------------------
species_x ='Steelhead'

# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

# -------------- habitat attributes from limiting factor output -----------
habitat_attributes = ['Temperature',
'Flow- Summer Base Flow',  'Substrate', 'Cover- Wood',
'Pools- All Pools', 'Off-Channel- Floodplain',
'Off-Channel- Side-Channels', 'Channel Stability',
'Bank Stability', 'Riparian']

# ---------------- data frame to combine with projects --------------
projects_all_reaches_Steelhead_df = pd.DataFrame([])

# -------------- data frame to combine within life stage data --------
life_stage_Steelhead_output = pd.DataFrame([])

# --------------------------------------------------------------------------
#                  Get Basic info for the reach
# --------------------------------------------------------------------------

life_stage_output_x = Limiting_Factor_Steelhead_ALL_Reaches[['Species', 'Life_Stage', 'Subbasin', 'Assessment Unit','ReachName', 'Habitat Attribute', 'Habitat Attribute Score']]
life_stage_output_x.columns = ['Species', 'Life_Stage','Subbasin', 'Assessment_Unit','ReachName','Habitat_Attribute', 'Limiting_Factor_Score']
life_stage_Steelhead_output = life_stage_Steelhead_output.append(life_stage_output_x)

# --------------------------------------------------------------------------
#               Input for Projects Processing
# --------------------------------------------------------------------------

# ----------------- Reach Name for this Specific reach -----------
reach_name_x = Limiting_Factor_Steelhead_ALL_Reaches['ReachName']

# ---------------- Idenfity habitat attribute for this reach ---------------
habitat_attribute_x = Limiting_Factor_Steelhead_ALL_Reaches['Habitat Attribute']

# ---------------- Idenfity habitat attribute for this reach ---------------
life_stage_x = Limiting_Factor_Steelhead_ALL_Reaches['Life_Stage']

# -------------- create Data Frame with reach and habitat attribute -------
reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x, life_stage_x])
reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

# ------------ identify habitat attribute names connected to Action Category/Type --------
crosswalk_habitat_attribute_output_x = pd.DataFrame([])
for habitat_attribute_x in reach_habitat_attribute_data_frame['Habitat Attribute']: 
    crosswalk_habitat_attribute_output_x = crosswalk_habitat_attribute_output_x.append(Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]) 
crosswalk_habitat_attribute_output_x.columns = ['Habitat_Attribute']

# ---------------------------------------------------------------------------
#      identify action categories, types, and projects (when project is linked via crosswalk to habitat attributes)
# ---------------------------------------------------------------------------

for i in range(0,reach_habitat_attribute_data_frame.shape[0]):
    crosswalk_x = crosswalk_habitat_attribute_output_x.iloc[i]
    reach_name_x  = reach_habitat_attribute_data_frame['ReachName'].iloc[i]
    life_stage_x = life_stage_Steelhead_output.iloc[i]
    output_x = output_habitat_actions_categories_FUNCTION(crosswalk_x, reach_name_x, life_stage_x)
    projects_all_reaches_Steelhead_df = projects_all_reaches_Steelhead_df.append(output_x)


### ------------------------------------------------------------------------------------------------------------------
###          Output Data
### ------------------------------------------------------------------------------------------------------------------

In [430]:
projects_all_reaches_Steelhead_df.to_excel(output_location_COMPLETE_Steelhead_Limiting_Factors, index=False)

In [431]:
# --------------------------------------------------------------
#        Initial Variables 
# --------------------------------------------------------------
species_x ='Spring Chinook'

# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

# -------------- habitat attributes from limiting factor output -----------
habitat_attributes = ['Temperature',
'Flow- Summer Base Flow',  'Substrate', 'Cover- Wood',
'Pools- All Pools', 'Off-Channel- Floodplain',
'Off-Channel- Side-Channels', 'Channel Stability',
'Bank Stability', 'Riparian']

# ---------------- data frame to combine with projects --------------
projects_all_reaches_Spring_Chinook_df = pd.DataFrame([])

# -------------- data frame to combine within life stage data --------
life_stage_Spring_Chinook_output = pd.DataFrame([])

# --------------------------------------------------------------------------
#                  Get Basic info for the reach
# --------------------------------------------------------------------------

life_stage_output_x = Limiting_Factor_Spring_Chinook_ALL_Reaches[['Species', 'Life_Stage', 'Subbasin', 'Assessment Unit','ReachName', 'Habitat Attribute', 'Habitat Attribute Score']]
life_stage_output_x.columns = ['Species', 'Life_Stage','Subbasin', 'Assessment_Unit','ReachName','Habitat_Attribute', 'Limiting_Factor_Score']
life_stage_Spring_Chinook_output = life_stage_Spring_Chinook_output.append(life_stage_output_x)

# --------------------------------------------------------------------------
#               Input for Projects Processing
# --------------------------------------------------------------------------

# ----------------- Reach Name for this Specific reach -----------
reach_name_x = Limiting_Factor_Spring_Chinook_ALL_Reaches['ReachName']

# ---------------- Idenfity habitat attribute for this reach ---------------
habitat_attribute_x = Limiting_Factor_Spring_Chinook_ALL_Reaches['Habitat Attribute']

# ---------------- Idenfity habitat attribute for this reach ---------------
life_stage_x = Limiting_Factor_Spring_Chinook_ALL_Reaches['Life_Stage']

# -------------- create Data Frame with reach and habitat attribute -------
reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x, habitat_attribute_x, life_stage_x])
reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

# ------------ identify habitat attribute names connected to Action Category/Type --------
crosswalk_habitat_attribute_output_x = pd.DataFrame([])
for habitat_attribute_x in reach_habitat_attribute_data_frame['Habitat Attribute']: 
    crosswalk_habitat_attribute_output_x = crosswalk_habitat_attribute_output_x.append(Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]) 
crosswalk_habitat_attribute_output_x.columns = ['Habitat_Attribute']

# ---------------------------------------------------------------------------


In [433]:
i = 117
crosswalk_x = crosswalk_habitat_attribute_output_x.iloc[i]
reach_name_x  = reach_habitat_attribute_data_frame['ReachName'].iloc[i]
life_stage_x = life_stage_Spring_Chinook_output.iloc[i]
output_habitat_actions_categories_FUNCTION(crosswalk_x, reach_name_x, life_stage_x)




,Species,Life_Stage,Subbasin,Assessment_Unit,ReachName,Habitat_Attribute,Limiting_Factor_Score,Action Category,Action Type,Project_present_yes_no,Project
0,Spring Chinook,Holding and Maturation,Methow,Chewuch River-Doe Creek,Chewuch River Doe 02,Temperature- Adult Holding,5,NA,NA,no,NA


In [382]:
crosswalk_habitat_attribute_x

Habitat_Attribute    Pools- All Pools
Name: 0, dtype: object

In [330]:
action_type_x

'Rock weirs, Boulder placement, LWD placement and engineered log jams, Levee modification: removal, setback, breach, Remove or relocate floodplain infrastructure, Restoration of floodplain topography and vegetation, Floodplain construction'

In [317]:

# --------------- if there is no Projects in this specific reach --------------
if Projects_List_Specific_Reach_x.shape[0] == 0:
    output_row_x = pd.DataFrame([life_stage_output_x['Species'], life_stage_output_x['Life_Stage'], life_stage_output_x['Subbasin'],
                                 life_stage_output_x['Assessment_Unit'], life_stage_output_x['ReachName'], life_stage_output_x['Habitat_Attribute'], 
                                 life_stage_output_x['Limiting_Factor_Score'], action_category_x, action_type_x,'no', 'NA'])
    output_row_x = output_row_x.transpose()
    output_row_x.columns = [[life_stage_output.columns[0],life_stage_output.columns[1],life_stage_output.columns[2],
                             life_stage_output.columns[3],life_stage_output.columns[4],life_stage_output.columns[5],
                              life_stage_output.columns[6],
                             'Action Category','Action Type','Project_present_yes_no','Project']]

# -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute ----
elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

    output_row_x = pd.DataFrame([life_stage_output_x['Species'], life_stage_output_x['Life_Stage'], life_stage_output_x['Subbasin'],
                                 life_stage_output_x['Assessment_Unit'], life_stage_output_x['ReachName'], life_stage_output_x['Habitat_Attribute'], 
                                 life_stage_output_x['Limiting_Factor_Score'],  
                                 action_category_x,action_type_x, 'no', 'NA'])
    output_row_x = output_row_x.transpose()
    output_row_x.columns = [[life_stage_output.columns[0],life_stage_output.columns[1],life_stage_output.columns[2],
                             life_stage_output.columns[3],life_stage_output.columns[4],life_stage_output.columns[5],
                             life_stage_output.columns[6],
                             'Action Category','Action Type','Project_present_yes_no','Project']]

# ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
else:
    #print('-----------------reaches with projects for impaired habitat attribute-------------')
    #print(reach_habitat_attribute_data_frame)

    # ----------------- Add the habitat attribute name to the data frame ----------------

    # --------------------------------------------------------------------------
    #                      For Action Type data frame 
    # --------------------------------------------------------------------------
    # reach_name_x, life_stage_output_x
    reach_habitat_attribute_data_frame_for_output_x = [reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1)
    reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute','Life_Stage']
    reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index

    # ------------------- add Habitat Attribute name -----------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
    # ----------------------- add life stage --------------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

    # --------------------------------------------------------------------------
    #                      For Action Category data frame 
    # --------------------------------------------------------------------------
    #reach_habitat_attribute_data_frame_for_output_x = [reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1)
    #reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute', 'Life_Stage']
    #reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index

    # ------------------- add Habitat Attribute name -----------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

    # ----------------------- add life stage --------------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

    # --------------------------------------------------------------------------
    #           Save output for Action Category and Action Types
    # --------------------------------------------------------------------------

    # -------- action category - if more than one, combine all --------
    action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]

    # -------- action type - if more than one, combine all --------
    action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
    action_category_x = ','.join(action_category_x)


    # ------------- action (project) descriptions ----------
    project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
    project_x = ','.join(project_x)

    output_row_x = pd.DataFrame([life_stage_output_x['Species'], life_stage_output_x['Life_Stage'], life_stage_output_x['Subbasin'],
                                 life_stage_output_x['Assessment_Unit'], life_stage_output_x['ReachName'], life_stage_output_x['Limiting_Factor_Score'],
                                life_stage_output_x['Limiting_Factor_Score'],  
                                 action_category_x, action_type_x,'yes', project_x])
    output_row_x = output_row_x.transpose()
    output_row_x.columns = [[life_stage_output.columns[0],life_stage_output.columns[1],life_stage_output.columns[2],
                             life_stage_output.columns[3],life_stage_output.columns[4],life_stage_output.columns[5],
                             life_stage_output.columns[6],
                             'Action Category','Action Type','Project_present_yes_no','Project']]

,ProjectID,Reach Assessment,ProjectName,Sub-Unit,Reach_ID_in_assessment,Reach_UCSRB,Action_Category,Action_Type,Action_Description,Matrix_No_Acres_Floodplain,Pieces_Wood_Low,Pieces_Wood_High,No_Boulders,Project_River_Length_feet,Project_River_Length_miles,Acres,VSP Parameters Addressed,Geomorphic Potential,Tier,Track,Rehabilitation Response Potential,Priority,Project Rank,Total Benefit Score
852,354,Chewuch River Reach Assessment,NaN,IZ-1,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 6.65C: Re-establish channel LWD dynamics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,355,Chewuch River Reach Assessment,NaN,IZ-1,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 6.27L: Re-establish channel LWD dynamics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,357,Chewuch River Reach Assessment,NaN,IZ-1,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 6.1C: Re-establish channel LWD dynamics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,360,Chewuch River Reach Assessment,NaN,IZ-1,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 6.95R: LWD enhancement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,361,Chewuch River Reach Assessment,NaN,IZ-1,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 6.39R: LWD enhancement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
859,362,Chewuch River Reach Assessment,NaN,IZ-1,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 5.9L: LWD enhancement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
860,367,Chewuch River Reach Assessment,NaN,IZ-2,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 5.69R: LWD enhancement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,368,Chewuch River Reach Assessment,NaN,IZ-2,Reach C2b,Chewuch River Pearrygin 05,Channel Complexity Restoration,LWD placement and engineered log jams,RM 5.69L: LWD enhancement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [312]:
[reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1)

[                    ReachName       Habitat Attribute       Life_Stage
 0        Chewuch River Doe 01  Flow- Summer Base Flow  Adult Migration
 1        Chewuch River Doe 02  Flow- Summer Base Flow  Adult Migration
 2        Chewuch River Doe 03  Flow- Summer Base Flow  Adult Migration
 3        Chewuch River Doe 04  Flow- Summer Base Flow  Adult Migration
 4        Chewuch River Doe 05  Flow- Summer Base Flow  Adult Migration
 ...                       ...                     ...              ...
 4498  Entiat River Preston 04    Predators- Juveniles            Smolt
 4499  Entiat River Preston 05    Predators- Juveniles            Smolt
 4500         Fish Lake Run 01    Predators- Juveniles            Smolt
 4501           Goose Creek 01    Predators- Juveniles            Smolt
 4502         Hancock Creek 01    Predators- Juveniles            Smolt
 
 [4503 rows x 3 columns],
                     ReachName       Habitat Attribute       Life_Stage
 0        Chewuch River Doe 01  Flow

(4503,)

In [ ]:
# ----------- loop through habitat attributes to verify if habitat attribute is in the crosswalk -------------
for habitat_attribute_x in reach_habitat_attribute_data_frame['Habitat Attribute']:

    # -------- if habitat attribute is in crosswalk list -------
    boolean_output = Habitat_Attribute_Names_Crosswalk_df == habitat_attribute_x
    boolean_T_F = boolean_output.any(axis = 'columns')
    
    if boolean_T_F.any():

        # ------------ identify habitat attribute names connected to Action Category/Type --------
        crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

        # ------------- identify Action Category and Action Type related to impaired habitat attribute
        pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

        # ---------------------- Only get Projects developed for this specific reach ---------
        Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

        # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

        # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

        # -------- action category - if more than one, combine all --------
        action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
        action_category_x = ','.join(action_category_x)
        # -------- action type - if more than one, combine all --------
        action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
        action_type_x = ','.join(action_type_x) 

        # --------------- if there is no Projects in this specific reach --------------
        if Projects_List_Specific_Reach_x.shape[0] == 0:
            empty_row_x = pd.DataFrame([action_category_x, action_type_x,'no', 'NA'])
            empty_row_x = empty_row_x.transpose()
            empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
            projects_all_reaches_df = projects_all_reaches_df.append(empty_row_x)

        # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----
        elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

            empty_row_x = pd.DataFrame([action_category_x,action_type_x, 'no', 'NA'])
            empty_row_x = empty_row_x.transpose()
            empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
            projects_all_reaches_df = projects_all_reaches_df.append(empty_row_x)

        # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
        else:
            #print('-----------------reaches with projects for impaired habitat attribute-------------')
            #print(reach_habitat_attribute_data_frame)

            # ----------------- Add the habitat attribute name to the data frame ----------------

            # --------------------------------------------------------------------------
            #                      For Action Type data frame 
            # --------------------------------------------------------------------------

            reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
            reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute','Life_Stage']
            reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index

            # ------------------- add Habitat Attribute name -----------
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
            # ----------------------- add life stage --------------
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

            # --------------------------------------------------------------------------
            #                      For Action Category data frame 
            # --------------------------------------------------------------------------
            reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
            reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute', 'Life_Stage']
            reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index

            # ------------------- add Habitat Attribute name -----------
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

            # ----------------------- add life stage --------------
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

            # --------------------------------------------------------------------------
            #           Save output for Action Category and Action Types
            # --------------------------------------------------------------------------

            # -------- action category - if more than one, combine all --------
            action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]

            # -------- action type - if more than one, combine all --------
            action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
            action_category_x = ','.join(action_category_x)


            # ------------- action (project) descriptions ----------
            project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
            project_x = ','.join(project_x)

            reach_row_x = pd.DataFrame([action_category_x, action_type_x,'yes', project_x])
            reach_row_x = reach_row_x.transpose()
            reach_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
            projects_all_reaches_df = projects_all_reaches_df.append(reach_row_x)

        # --------------------------------------------------------------------------
        #           IF the habitat attribute is not in the crosswalk
        # --------------------------------------------------------------------------

    else:
        empty_row_x = pd.DataFrame(['NA','NA','no', 'NA'])
        empty_row_x = empty_row_x.transpose()
        empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
        projects_all_reaches_df = projects_all_reaches_df.append(empty_row_x)



In [51]:
projects_all_reaches_df.shape

(10, 4)

In [ ]:
# ------------------------------ OUTPUT to an Excel spreadsheet  ------------------------- 
# ---------------- combine the data frames --------------
projects_Master_data_frame = cbind(projects_all_reaches_df, life_stage_output)
Projects_for_impaired_habitat_attributes_updated.to_excel(output_location_COMPLETE_Spring_Chinook_Limiting_Factors, index = False)

# --------------------------------------------------------------
#       Steelhead
# --------------------------------------------------------------

In [ ]:
    # List of columns: Species, Life_Stage, Basin, Assessment_Unit, Habitat_Attribute, Limiting Factor Score,
    #                 Action Category,  Action Type, Project Present (yes/no), Project
    
# --------------------------------------------------------------
#        Initial Variables 
# --------------------------------------------------------------
species_x ='Steelhead'
Output_all_reaches_limiting_factors_projects = pd.DataFrame([])

# ------- Use Steelhead data ----------
criteria_data_adult_migration = Limiting_Factor_Steelhead_Adult_Migration
criteria_data_holding_and_maturation = Limiting_Factor_Steelhead_Holding_and_Maturation
criteria_data_spawning = Limiting_Factor_Steelhead_Spawning
criteria_data_fry = Limiting_Factor_Steelhead_Fry
criteria_data_summer_rearing = Limiting_Factor_Steelhead_Summer_Rearing
criteria_data_winter_rearing = Limiting_Factor_Steelhead_Winter_Rearing
criteria_data_smolt = Limiting_Factor_Steelhead_Smolt

# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

# -------------- habitat attributes from limiting factor output -----------
habitat_attributes = ['Temperature',
       'Flow- Summer Base Flow',  'Substrate', 'Cover- Wood',
       'Pools- All Pools', 'Off-Channel- Floodplain',
       'Off-Channel- Side-Channels', 'Channel Stability',
       'Bank Stability', 'Riparian']

# ---------------- data frame to combine with projects --------------
projects_all_reaches_df = pd.DataFrame([])

# -------------- data frame to combine within life stage data --------
life_stage_output = pd.DataFrame([])

# --------------------------------------------------------------------------
#                    Iterate Across All Life Stages 
# --------------------------------------------------------------------------

    for criteria_data in (criteria_data_adult_migration, criteria_data_holding_and_maturation, criteria_data_spawning, criteria_data_fry, 
                      criteria_data_summer_rearing, criteria_data_winter_rearing,  criteria_data_smolt):
    
        # --------------------------------------------------------------------------
        #                    Iterate Across Rows/Reaches in each Life Stage
        # --------------------------------------------------------------------------

        
        for index, row in criteria_data.iterrows():

            # --------------------------------------------------------------------------
            #                     Add Initial/Basic Values
            # --------------------------------------------------------------------------

            # ------------- make row into data frame ------------
            row = pd.DataFrame(row).transpose()
            # ----------- add Species, Life Stage, Subbasin, Assessment Unit, habitat Attribute, Limtiting Factor Score ------
            row_output= row[['Species', 'Life_Stage', 'Subbasin', 'Assessment Unit','ReachName', 'Habitat Attribute', 'Habitat Attribute Score']]
            row_output.columns = ['Species', 'Life_Stage','Subbasin', 'Assessment_Unit','ReachName','Habitat_Attribute', 'Limiting_Factor_Score']
            # ----------- combine with summary data frame ---------------
            life_stage_output = life_stage_output.append(row_output)

        # --------------------------------------------------------------------------
        #                     Add Action Categories, Types, and Projects etc.
        # --------------------------------------------------------------------------


        for index, row in life_stage_output.iterrows():

            # ----------------- Reach Name for this Specific reach -----------
            reach_name_x = row['ReachName']

            # ---------------- Idenfity habitat attribute for this reach ---------------
            habitat_attribute_x = row['Habitat_Attribute']

            # ---------------- Idenfity habitat attribute for this reach ---------------
            life_stage_x = row['Life_Stage']

            #print(reach_name_x + habitat_attribute_x)

            # -------------- create Data Frame with reach and habitat attribute -------
            reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x, life_stage_x])
            reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

            print(habitat_attribute_x)
            # ----------- loop through habitat attributes to verify if habitat attribute is in the crosswalk -------------
            for habitat_attribute_x in habitat_attributes:

                # -------- if habitat attribute is in crosswalk list -------
                boolean_output = Habitat_Attribute_Names_Crosswalk_df == habitat_attribute_x
                boolean_T_F = boolean_output.any(axis = 'columns')
                if boolean_T_F.any():

                    # ------------ identify habitat attribute names connected to Action Category/Type --------
                    crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

                    # ------------- identify Action Category and Action Type related to impaired habitat attribute
                    pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

                    # ---------------------- Only get Projects developed for this specific reach ---------
                    Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

                    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
                    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

                    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
                    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

                    # -------- action category - if more than one, combine all --------
                    action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
                    action_category_x = ','.join(action_category_x)
                    # -------- action type - if more than one, combine all --------
                    action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
                    action_type_x = ','.join(action_type_x) 

                    # --------------- if there is no Projects in this specific reach --------------
                    if Projects_List_Specific_Reach_x.shape[0] == 0:
                        empty_row_x = pd.DataFrame([action_category_x, action_type_x,'no', 'NA'])
                        empty_row_x = empty_row_x.transpose()
                        empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                        projects_all_reaches_df = projects_all_reaches_df.append(empty_row_x)

                    # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----



                    elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

                        empty_row_x = pd.DataFrame([action_category_x,action_type_x, 'no', 'NA'])
                        empty_row_x = empty_row_x.transpose()
                        empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                        projects_all_reaches_df = projects_all_reaches_df.append(empty_row_x)

                    # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
                    else:
                        #print('-----------------reaches with projects for impaired habitat attribute-------------')
                        #print(reach_habitat_attribute_data_frame)

                        # ----------------- Add the habitat attribute name to the data frame ----------------

                        # --------------------------------------------------------------------------
                        #                      For Action Type data frame 
                        # --------------------------------------------------------------------------

                        reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
                        reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute','Life_Stage']
                        reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index

                        # ------------------- add Habitat Attribute name -----------
                        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
                        # ----------------------- add life stage --------------
                        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

                        # --------------------------------------------------------------------------
                        #                      For Action Category data frame 
                        # --------------------------------------------------------------------------
                        reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
                        reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute', 'Life_Stage']
                        reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index

                        # ------------------- add Habitat Attribute name -----------
                        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

                        # ----------------------- add life stage --------------
                        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

                        # --------------------------------------------------------------------------
                        #           Save output for Action Category and Action Types
                        # --------------------------------------------------------------------------

                        # -------- action category - if more than one, combine all --------
                        action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]

                        # -------- action type - if more than one, combine all --------
                        action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
                        action_category_x = ','.join(action_category_x)


                        # ------------- action (project) descriptions ----------
                        project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
                        project_x = ','.join(project_x)

                        reach_row_x = pd.DataFrame([action_category_x, action_type_x,'yes', project_x])
                        reach_row_x = reach_row_x.transpose()
                        reach_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                        projects_all_reaches_df = projects_all_reaches_df.append(reach_row_x)

                    # --------------------------------------------------------------------------
                    #           IF the habitat attribute is not in the crosswalk
                    # --------------------------------------------------------------------------

                else:
                    empty_row_x = pd.DataFrame(['NA','NA','no', 'NA'])
                    empty_row_x = empty_row_x.transpose()
                    empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                    projects_all_reaches_df = projects_all_reaches_df.append(empty_row_x)


    

In [64]:
projects_all_reaches_df.head(10)

,0,1,2
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt
0,Alder Creek 01,Predators- Juveniles,Smolt


In [19]:
criteria_data = criteria_data_holding_and_maturation
criteria_data.shape

(806, 10)

In [20]:

# --------------------------------------------------------------------------
#                    Iterate Across Rows/Reaches in each Life Stage
# --------------------------------------------------------------------------

# -------------- data frame to combine within life stage data --------
life_stage_output = pd.DataFrame([])

for index, row in criteria_data.iterrows():

    # --------------------------------------------------------------------------
    #                     Add Initial/Basic Values
    # --------------------------------------------------------------------------

    # ------------- make row into data frame ------------
    row = pd.DataFrame(row).transpose()
    # ----------- add Species, Life Stage, Subbasin, Assessment Unit, habitat Attribute, Limtiting Factor Score ------
    row_output= row[['Species', 'Life_Stage', 'Subbasin', 'Assessment Unit','ReachName', 'Habitat Attribute', 'Habitat Attribute Score']]
    row_output.columns = ['Species', 'Life_Stage','Subbasin', 'Assessment_Unit','ReachName','Habitat_Attribute', 'Limiting_Factor_Score']
    # ----------- combine with summary data frame ---------------
    life_stage_output = life_stage_output.append(row_output)




In [21]:
life_stage_output

,Species,Life_Stage,Subbasin,Assessment_Unit,ReachName,Habitat_Attribute,Limiting_Factor_Score
0,Spring Chinook,Holding and Maturation,Methow,Chewuch River-Doe Creek,Chewuch River Doe 01,Cover- Undercut Banks,1
1,Spring Chinook,Holding and Maturation,Methow,Chewuch River-Doe Creek,Chewuch River Doe 01,Temperature- Adult Holding,5
2,Spring Chinook,Holding and Maturation,Methow,Chewuch River-Doe Creek,Chewuch River Doe 01,Cover- Wood,5
3,Spring Chinook,Holding and Maturation,Methow,Chewuch River-Doe Creek,Chewuch River Doe 01,Harassment,1
4,Spring Chinook,Holding and Maturation,Methow,Chewuch River-Doe Creek,Chewuch River Doe 01,Pools- All Pools,3
...,...,...,...,...,...,...,...
801,Spring Chinook,Holding and Maturation,Wenatchee,Wenatchee River-Beaver Creek,Wenatchee River Beaver 13,Cover- Undercut Banks,3
802,Spring Chinook,Holding and Maturation,Wenatchee,Wenatchee River-Beaver Creek,Wenatchee River Beaver 13,Pools- Deep Pools,1
803,Spring Chinook,Holding and Maturation,Wenatchee,Wenatchee River-Beaver Creek,Wenatchee River Beaver 13,Cover- Boulders,1
804,Spring Chinook,Holding and Maturation,Wenatchee,Wenatchee River-Beaver Creek,Wenatchee River Beaver 13,Flow- Summer Base Flow,3


In [54]:
    
# ------------------------------------------------------------------------------
#      Step 3:  Identify Action Types/Categories linked to impacted Habitat Attributes in each impaired reach (based on criteria in Step 1) 
# ------------------------------------------------------------------------------

# ---------------- data frame to combine with projects --------------
projects_all_reaches_df = pd.DataFrame([])

for index, row in life_stage_output.iterrows():

    # ----------------- Reach Name for this Specific reach -----------
    reach_name_x = row['ReachName']

    # ---------------- Idenfity habitat attribute for this reach ---------------
    habitat_attribute_x = row['Habitat_Attribute']

    # ---------------- Idenfity habitat attribute for this reach ---------------
    life_stage_x = row['Life_Stage']

    print(reach_name_x + habitat_attribute_x)
    
    # -------------- create Data Frame with reach and habitat attribute -------
    reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x, life_stage_x])
    reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

    print(habitat_attribute_x)
    # ----------- loop through habitat attributes to verify if habitat attribute is in the crosswalk -------------
    for habitat_attribute_x in habitat_attributes:

        # -------- if habitat attribute is in crosswalk list -------
        boolean_output = Habitat_Attribute_Names_Crosswalk_df == habitat_attribute_x
        boolean_T_F = boolean_output.any(axis = 'columns')
        if boolean_T_F.any():

            # ------------ identify habitat attribute names connected to Action Category/Type --------
            crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

            # ------------- identify Action Category and Action Type related to impaired habitat attribute
            pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

            # ---------------------- Only get Projects developed for this specific reach ---------
            Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

            # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
            Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

            # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
            Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

            # -------- action category - if more than one, combine all --------
            action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
            action_category_x = ','.join(action_category_x)
            # -------- action type - if more than one, combine all --------
            action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
            action_type_x = ','.join(action_type_x) 

            # --------------- if there is no Projects in this specific reach --------------
            if Projects_List_Specific_Reach_x.shape[0] == 0:
                empty_row_x = pd.DataFrame([action_category_x, action_type_x,'no', 'NA'])
                empty_row_x = empty_row_x.transpose()
                empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                projects_all_reaches_df = projects_all_reaches_df.append(reach_habitat_attribute_data_frame)

            # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----
            
                            

            elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:

                empty_row_x = pd.DataFrame([action_category_x,action_type_x, 'no', 'NA'])
                empty_row_x = empty_row_x.transpose()
                empty_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                projects_all_reaches_df = projects_all_reaches_df.append(reach_habitat_attribute_data_frame)

            # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
            else:
                #print('-----------------reaches with projects for impaired habitat attribute-------------')
                #print(reach_habitat_attribute_data_frame)

                # ----------------- Add the habitat attribute name to the data frame ----------------

                # --------------------------------------------------------------------------
                #                      For Action Type data frame 
                # --------------------------------------------------------------------------

                reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
                reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute','Life_Stage']
                reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index

                # ------------------- add Habitat Attribute name -----------
                Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
                # ----------------------- add life stage --------------
                Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

                # --------------------------------------------------------------------------
                #                      For Action Category data frame 
                # --------------------------------------------------------------------------
                reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
                reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute', 'Life_Stage']
                reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index

                # ------------------- add Habitat Attribute name -----------
                Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

                # ----------------------- add life stage --------------
                Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

                # --------------------------------------------------------------------------
                #           Save output for Action Category and Action Types
                # --------------------------------------------------------------------------
                
                # -------- action category - if more than one, combine all --------
                action_category_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Category'][:]]
                                
                # -------- action type - if more than one, combine all --------
                action_type_x = [''.join(col).strip() for col in pathways_habitat_attributes_x['Action Type'][:]]
                action_category_x = ','.join(action_category_x)
                
            
                # ------------- action (project) descriptions ----------
                project_x = [''.join(col).strip() for col in Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Action_Description'][:]]
                project_x = ','.join(project_x)
                
                reach_row_x = pd.DataFrame([action_category_x, action_type_x,'yes', project_x])
                reach_row_x = reach_row_x.transpose()
                reach_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]
                projects_all_reaches_df = projects_all_reaches_df.append(reach_habitat_attribute_data_frame)

            # --------------------------------------------------------------------------
            #           IF the habitat attribute is not in the crosswalk
            # --------------------------------------------------------------------------

        else:
            empty_row_x = pd.DataFrame([['NA','NA','no', 'NA']])
            empty_row_x.columns = ['Action Category','Action Type','Project_present_yes_no','Project']
            projects_all_reaches_df = projects_all_reaches_df.append(reach_habitat_attribute_data_frame)


Chewuch River Doe 01Cover- Undercut Banks
Cover- Undercut Banks
Chewuch River Doe 01Temperature- Adult Holding
Temperature- Adult Holding


<ipython-input-54-678f04657c8e>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
<ipython-input-54-678f04657c8e>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x
<ipython-input-54-678f04657c8e>:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

Chewuch River Doe 01Cover- Wood
Cover- Wood
Chewuch River Doe 01Harassment
Harassment
Chewuch River Doe 01Pools- All Pools
Pools- All Pools
Chewuch River Doe 01Flow- Summer Base Flow
Flow- Summer Base Flow
Chewuch River Doe 01Pools- Deep Pools
Pools- Deep Pools
Chewuch River Doe 02Temperature- Adult Holding
Temperature- Adult Holding
Chewuch River Doe 02Harassment
Harassment
Chewuch River Doe 02Pools- All Pools
Pools- All Pools
Chewuch River Doe 02Flow- Summer Base Flow
Flow- Summer Base Flow
Chewuch River Doe 02Cover- Undercut Banks
Cover- Undercut Banks
Chewuch River Doe 02Pools- Deep Pools
Pools- Deep Pools
Chewuch River Doe 02Cover- Wood
Cover- Wood
Chewuch River Doe 03Flow- Summer Base Flow
Flow- Summer Base Flow
Chewuch River Doe 03Pools- Deep Pools
Pools- Deep Pools
Chewuch River Doe 03Cover- Undercut Banks
Cover- Undercut Banks
Chewuch River Doe 03Pools- All Pools
Pools- All Pools
Chewuch River Doe 03Cover- Wood
Cover- Wood
Chewuch River Doe 03Harassment
Harassment
Chewuch Rive

Chewuch River Thirtymile 01Cover- Wood
Cover- Wood
Chewuch River Thirtymile 01Pools- Deep Pools
Pools- Deep Pools
Chewuch River Thirtymile 01Cover- Boulders
Cover- Boulders
Chewuch River Thirtymile 01Pools- All Pools
Pools- All Pools
Chewuch River Thirtymile 01Harassment
Harassment
Chewuch River Thirtymile 01Temperature- Adult Holding
Temperature- Adult Holding
Chewuch River Thirtymile 01Cover- Undercut Banks
Cover- Undercut Banks
Chewuch River Thirtymile 02Temperature- Adult Holding
Temperature- Adult Holding
Chewuch River Thirtymile 02Flow- Summer Base Flow
Flow- Summer Base Flow
Chewuch River Thirtymile 02Cover- Undercut Banks
Cover- Undercut Banks
Chewuch River Thirtymile 02Cover- Wood
Cover- Wood
Chewuch River Thirtymile 02Pools- All Pools
Pools- All Pools
Chewuch River Thirtymile 02Pools- Deep Pools
Pools- Deep Pools
Chewuch River Thirtymile 02Harassment
Harassment
Chewuch River Thirtymile 02Cover- Boulders
Cover- Boulders
Chewuch River Thirtymile 03Cover- Wood
Cover- Wood
Chewuc

Entiat River Potato 03Cover- Boulders
Cover- Boulders
Entiat River Potato 04Temperature- Adult Holding
Temperature- Adult Holding
Entiat River Potato 04Harassment
Harassment
Entiat River Potato 04Flow- Summer Base Flow
Flow- Summer Base Flow
Entiat River Potato 04Cover- Boulders
Cover- Boulders
Entiat River Potato 04Cover- Undercut Banks
Cover- Undercut Banks
Entiat River Potato 05Cover- Boulders
Cover- Boulders
Entiat River Potato 05Flow- Summer Base Flow
Flow- Summer Base Flow
Entiat River Potato 05Pools- All Pools
Pools- All Pools
Entiat River Potato 05Harassment
Harassment
Entiat River Potato 05Temperature- Adult Holding
Temperature- Adult Holding
Entiat River Potato 05Cover- Undercut Banks
Cover- Undercut Banks
Entiat River Potato 05Pools- Deep Pools
Pools- Deep Pools
Entiat River Potato 05Cover- Wood
Cover- Wood
Entiat River Potato 06Pools- All Pools
Pools- All Pools
Entiat River Potato 06Cover- Undercut Banks
Cover- Undercut Banks
Entiat River Potato 06Flow- Summer Base Flow
Flo

Methow River Fawn 08Pools- Deep Pools
Pools- Deep Pools
Methow River Fawn 08Cover- Wood
Cover- Wood
Methow River Fawn 08Harassment
Harassment
Methow River Fawn 09Temperature- Adult Holding
Temperature- Adult Holding
Methow River Fawn 09Pools- All Pools
Pools- All Pools
Methow River Fawn 09Cover- Boulders
Cover- Boulders
Methow River Fawn 09Flow- Summer Base Flow
Flow- Summer Base Flow
Methow River Fawn 09Cover- Wood
Cover- Wood
Methow River Fawn 09Harassment
Harassment
Methow River Fawn 09Pools- Deep Pools
Pools- Deep Pools
Methow River Fawn 09Cover- Undercut Banks
Cover- Undercut Banks
Methow River Fawn 10Harassment
Harassment
Methow River Fawn 10Cover- Wood
Cover- Wood
Methow River Fawn 10Pools- Deep Pools
Pools- Deep Pools
Methow River Fawn 10Cover- Boulders
Cover- Boulders
Methow River Fawn 10Cover- Undercut Banks
Cover- Undercut Banks
Methow River Fawn 10Pools- All Pools
Pools- All Pools
Methow River Fawn 10Temperature- Adult Holding
Temperature- Adult Holding
Methow River Fawn 10

Stormy Creek 01Cover- Boulders
Cover- Boulders
Stormy Creek 01Pools- Deep Pools
Pools- Deep Pools
Stormy Creek 01Flow- Summer Base Flow
Flow- Summer Base Flow
Stormy Creek 01Harassment
Harassment
Stormy Creek 01Cover- Wood
Cover- Wood
Stormy Creek 01Temperature- Adult Holding
Temperature- Adult Holding
Stormy Creek 01Cover- Undercut Banks
Cover- Undercut Banks
Stormy Creek 01Pools- All Pools
Pools- All Pools
Twisp River Lower 01Pools- Deep Pools
Pools- Deep Pools
Twisp River Lower 01Cover- Wood
Cover- Wood
Twisp River Lower 01Harassment
Harassment
Twisp River Lower 01Temperature- Adult Holding
Temperature- Adult Holding
Twisp River Lower 01Flow- Summer Base Flow
Flow- Summer Base Flow
Twisp River Lower 01Cover- Boulders
Cover- Boulders
Twisp River Lower 01Pools- All Pools
Pools- All Pools
Twisp River Lower 01Cover- Undercut Banks
Cover- Undercut Banks
Twisp River Lower 02Cover- Wood
Cover- Wood
Twisp River Lower 02Pools- All Pools
Pools- All Pools
Twisp River Lower 02Pools- Deep Pools


Wenatchee River Beaver 02Pools- All Pools
Pools- All Pools
Wenatchee River Beaver 02Pools- Deep Pools
Pools- Deep Pools
Wenatchee River Beaver 02Cover- Undercut Banks
Cover- Undercut Banks
Wenatchee River Beaver 02Temperature- Adult Holding
Temperature- Adult Holding
Wenatchee River Beaver 02Flow- Summer Base Flow
Flow- Summer Base Flow
Wenatchee River Beaver 03Cover- Undercut Banks
Cover- Undercut Banks
Wenatchee River Beaver 03Pools- Deep Pools
Pools- Deep Pools
Wenatchee River Beaver 03Pools- All Pools
Pools- All Pools
Wenatchee River Beaver 03Cover- Wood
Cover- Wood
Wenatchee River Beaver 03Harassment
Harassment
Wenatchee River Beaver 03Temperature- Adult Holding
Temperature- Adult Holding
Wenatchee River Beaver 03Cover- Boulders
Cover- Boulders
Wenatchee River Beaver 03Flow- Summer Base Flow
Flow- Summer Base Flow
Wenatchee River Beaver 04Cover- Wood
Cover- Wood
Wenatchee River Beaver 04Pools- All Pools
Pools- All Pools
Wenatchee River Beaver 04Cover- Undercut Banks
Cover- Undercu

In [49]:
reach_row_x.columns = [['Action Category','Action Type','Project_present_yes_no','Project']]

868                                  RM 12.9C LWD enhancement
869    RM 12.3C LWD placement to enhance lateral connectivity
Name: Action_Description, dtype: object

In [52]:
reach_row_x

,0
0,"Channel Complexity Restoration,Channel Complexity Restoration,Channel Complexity Restoration"
1,"[Rock weirs, Boulder placement, LWD placement and engineered log jams]"
2,yes
3,"RM 12.9C LWD enhancement,RM 12.3C LWD placement to enhance lateral connectivity"


In [31]:
pd.DataFrame([action_category_x,action_type_x, 'no', 'NA'])

,0,1,2
0,Water Quality Improvement,Water Quality Improvement,Water Quality Improvement
1,Improve thermal refugia,Reduce or mitigate point-source effects,Upland vegetation treatment – management
2,n,o,None
3,N,A,None


In [936]:
pathways_habitat_attributes_x

,Action Type,Pathway,Habitat Attribute,Action Category,Notes,Reviewer Comments
7,Rock weirs,Limiting Factor/Habitat Quality,Cover- Wood,Channel Complexity Restoration,NaN,NaN
7,Boulder placement,Limiting Factor/Habitat Quality,Cover- Wood,Channel Complexity Restoration,NaN,NaN
7,LWD placement and engineered log jams,Limiting Factor/Habitat Quality,Cover- Wood,Channel Complexity Restoration,NaN,NaN
41,"Levee modification: removal, setback, breach",Limiting Factor,Cover- Wood,Floodplain Reconnection,Access floodplain wood,NaN
41,Remove or relocate floodplain infrastructure,Limiting Factor,Cover- Wood,Floodplain Reconnection,Access floodplain wood,NaN
41,Restoration of floodplain topography and vegetation,Limiting Factor,Cover- Wood,Floodplain Reconnection,Access floodplain wood,NaN
41,Floodplain construction,Limiting Factor,Cover- Wood,Floodplain Reconnection,Access floodplain wood,NaN


In [934]:
pd.DataFrame([pathways_habitat_attributes_x['Action Category'],               pathways_habitat_attributes_x['Action Type'],'yes', Projects_Action_Category_for_impaired_habitat_attributes['Action_Description']])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [926]:
reach_habitat_attribute_data_frame_for_output_x['Habitat Attribute']

KeyError: 'Habitat Attribute'

In [918]:
reach_habitat_attribute_data_frame_for_output_x =[reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1)
reach_habitat_attribute_data_frame_for_output_x.columns = [['ReachName','Habitat_Attribute','Life_Stage']]
reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index



AttributeError: 'list' object has no attribute 'columns'

In [915]:
reach_habitat_attribute_data_frame_for_output_x

,ReachName,Habitat_Attribute,Life_Stage
403,Chewuch River Doe 01,Cover- Undercut Banks,Holding and Maturation
868,Chewuch River Doe 01,Cover- Undercut Banks,Holding and Maturation
869,Chewuch River Doe 01,Cover- Undercut Banks,Holding and Maturation


In [791]:

# ------------------------------------------------------------------------------
#      Step 2: Identify impaired habitat attributes based on Habitat Quality criteria 
# ------------------------------------------------------------------------------

# -------------- data frame to add reaches with specific habitat attributes that are impaired -------
impaired_habitat_data_frame = pd.DataFrame([])

for criteria_data in (criteria_data_adult_migration, criteria_data_holding_and_maturation, criteria_data_spawning, criteria_data_fry, criteria_data_summer_rearing, criteria_data_winter_rearing,  criteria_data_smolt):

    # ---------------- generate unique habitat attributes for this life stage -----------
    habitat_attributes = criteria_data['Habitat Attribute'].sort_values().unique()

    # ----------- loop through habitat attributes -------------
    for habitat_attribute_x in habitat_attributes:

        # -------- if habitat attribute is in crosswalk list -------
        boolean_output = Habitat_Attribute_Names_Crosswalk_df == habitat_attribute_x
        boolean_T_F = boolean_output.any(axis = 'columns')
        if boolean_T_F.any():

            # ------------ generate criteria data just with habitat_attribute_x -----------
            criteria_data_habitat_attributre_x = criteria_data.loc[criteria_data['Habitat Attribute'].isin([habitat_attribute_x])]

            # ------------- identify all the reaches that are impaired based on the habitat_attribute_x attribute ------
            criteria_data_x = criteria_data_habitat_attributre_x[['ReachName', 'Assessment Unit', 'Subbasin','Habitat Attribute']][criteria_data_habitat_attributre_x['Habitat Attribute Score']>=habitat_quality_priority_score]
            # ------------------------- rename column names ----------
            criteria_data_x.columns  = ['ReachName', 'Assessment Unit', 'Subbasin', 'Habitat Attribute Score']
            # ------------------------- add habitat attribute name ----------
            criteria_data_x['Habitat_Attribute'] = habitat_attribute_x
            # ------------------------- add habitat attribute name ----------
            criteria_data_x['Life_Stage'] = criteria_data['Life_Stage'][0]
            # ----------------------- append to data frame -------------
            impaired_habitat_data_frame = impaired_habitat_data_frame.append(criteria_data_x)


            
# ------------------------------------------------------------------------------
#      Step 3:  Identify Action Types/Categories linked to impacted Habitat Attributes in each impaired reach (based on criteria in Step 1) 
# ------------------------------------------------------------------------------

reaches_not_present_in_projects = pd.DataFrame([])
reaches_present_no_projects = pd.DataFrame([])
Projects_Action_Type_for_impaired_habitat_attributes = pd.DataFrame([])
Projects_Action_Category_for_impaired_habitat_attributes = pd.DataFrame([])
Projects_for_impaired_habitat_attributes = pd.DataFrame([])
for index, row in impaired_habitat_data_frame.iterrows():

    # ----------------- Reach Name for this Specific reach -----------
    reach_name_x = row['ReachName']

    # ---------------- Idenfity habitat attribute for this reach ---------------
    habitat_attribute_x = row['Habitat_Attribute']

    # ---------------- Idenfity habitat attribute for this reach ---------------
    life_stage_x = row['Life_Stage']

    # -------------- create Data Frame with reach and habitat attribute -------
    reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x, life_stage_x])
    reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

    # ------------ identify habitat attribute names connected to Action Category/Type --------
    crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

    # ------------- identify Action Category and Action Type related to impaired habitat attribute
    pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

    # ---------------------- Only get Projects developed for this specific reach ---------
    Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

    #print(reach_name_x)
    #print(Projects_List_Specific_Reach_x.shape)
    #print(Projects_List_Specific_Reach_x_Habitat_Attribute.shape)

    # --------------- if there is no Projects in this specific reach --------------
    if Projects_List_Specific_Reach_x.shape[0] == 0:
        reaches_not_present_in_projects = reaches_not_present_in_projects.append(reach_habitat_attribute_data_frame)

    # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----
    elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:
        reaches_present_no_projects = reaches_present_no_projects.append(reach_habitat_attribute_data_frame)

    # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
    else:
        #print('-----------------reaches with projects for impaired habitat attribute-------------')
        #print(reach_habitat_attribute_data_frame)

        # ----------------- Add the habitat attribute name to the data frame ----------------

        # --------------------------------------------------------------------------
        #                      For Action Type data frame 
        # --------------------------------------------------------------------------

        reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
        reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute','Life_Stage']
        reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index

        # ------------------- add Habitat Attribute name -----------
        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
        # ----------------------- add life stage --------------
        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

        # --------------------------------------------------------------------------
        #                      For Action Category data frame 
        # --------------------------------------------------------------------------
        reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
        reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute', 'Life_Stage']
        reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index

        # ------------------- add Habitat Attribute name -----------
        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

        # ----------------------- add life stage --------------
        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x

        # --------------- save output to individual Action Category/Action Type data frames ---------
        Projects_Action_Type_for_impaired_habitat_attributes = Projects_Action_Type_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
        Projects_Action_Category_for_impaired_habitat_attributes = Projects_Action_Category_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)

        # --------------------------------------------------------------------------
        #           Save output for Action Category and Action Types
        # --------------------------------------------------------------------------

        Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
        Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)
        #print(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape)
        #print(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape)



# ------------------------- Collapse duplicated projects to one project, list multiple habitat attributes ----------
#Projects_for_impaired_habitat_attributes['Habitat_Attribute'] = Projects_for_impaired_habitat_attributes['Habitat_Attribute'].strip("[]")
project_data_frame = pd.DataFrame(Projects_for_impaired_habitat_attributes['ProjectID'])
duplicated_project_data_frame_x = project_data_frame.loc[project_data_frame.duplicated()].sort_values(by=['ProjectID'])



# ------- loop through projects to collapse duplicated projects ------- 
Projects_for_impaired_habitat_attributes_updated = pd.DataFrame([])
unique_Project_IDs_x = Projects_for_impaired_habitat_attributes.ProjectID.unique()
for projectID_x in unique_Project_IDs_x:

    # ---------- if a duplicated project --------
    if any(duplicated_project_data_frame_x.ProjectID.isin([projectID_x]) ): 
        Duplicated_Projects_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
        # ------- Habitat Attribute list for all the duplicated projects ----------
        Habitat_Attributes_duplicated_x = Duplicated_Projects_x.Habitat_Attribute
        Habitat_Attributes_duplicated_x = Habitat_Attributes_duplicated_x.unique()
        Habitat_Attributes_duplicated_x = str(np.stack(Habitat_Attributes_duplicated_x) )

        # ------------ Get a single row for all the duplicated projects ---------------
        #print('---------------- Projects_for_impaired_habitat_attributes  Life Stage')
        #print(Projects_for_impaired_habitat_attributes['Life_Stage'])
        row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])].iloc[0:1,]
        row_x.Habitat_Attribute = Habitat_Attributes_duplicated_x
        Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)
        #print(' ------------------ row_x - duplicated project --------------')
        #print(row_x['Life_Stage'])

    # ------- if project ID is not duplicated ----------
    else:
        row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
        Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)
        #print(' ------------------ row_x - not duplicated--------------')
        #print(row_x['Life_Stage'])

# ------------------------------------------------------------------------------
#      Step 4A: Generate output based on reaches
# ------------------------------------------------------------------------------

# --------- remove brackets from Habitat Attributes -------
Projects_for_impaired_habitat_attributes_updated['Habitat_Attribute'] = Projects_for_impaired_habitat_attributes_updated['Habitat_Attribute'].str.strip("[]")

# ------------------------------ OUTPUT to an Excel spreadsheet  -------------------------  
Projects_for_impaired_habitat_attributes_updated.to_excel(output_location_x, index = False)

# ------------------------------------------------------------------------------
#      Step 4B: Generate output for PWG meeting on September 15th
# ------------------------------------------------------------------------------
# List of columns: Species, Life_Stage, Basin, Assessment_Unit, Habitat_Attribute, Limiting Factor Score,
#                 Action Category,  Action Type, Project Present (yes/no), Project

# --------------- create data frame ----------------------
print(type(Projects_for_impaired_habitat_attributes_updated))
Output_all_reaches_data_frame = c()

<ipython-input-791-c9e8fc19ee30>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']
<ipython-input-791-c9e8fc19ee30>:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Life_Stage'] = life_stage_x
<ipython-input-791-c9e8fc19ee30>:115: SettingWithCopyWarning: 
A value is trying to be set on a copy 

NameError: name 'output_location_x' is not defined

In [792]:
criteria_data.head()

,Subbasin,Assessment Unit,AU Tier Spring CHN,SPCH_SmoltLSP,ReachName,SPCH_Smolt_Emigration,Species,Life_Stage,Habitat Attribute,Habitat Attribute Score


## ----------------------------------------------------------------------------------------------------------------------------
##           OLD Practice Code
## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
reaches_not_present_in_projects = pd.DataFrame([])
reaches_present_no_projects = pd.DataFrame([])
Projects_Action_Type_for_impaired_habitat_attributes = pd.DataFrame([])
Projects_Action_Category_for_impaired_habitat_attributes = pd.DataFrame([])
Projects_for_impaired_habitat_attributes = pd.DataFrame([])
for index, row in impaired_habitat_data_frame.iterrows():

    # ----------------- Reach Name for this Specific reach -----------
    reach_name_x = row['ReachName']

    # ---------------- Idenfity habitat attribute for this reach ---------------
    habitat_attribute_x = row['Habitat_Attribute']

    # -------------- create Data Frame with reach and habitat attribute -------
    reach_habitat_attribute_data_frame = pd.DataFrame([reach_name_x,habitat_attribute_x])
    reach_habitat_attribute_data_frame = reach_habitat_attribute_data_frame.transpose() # make it int data frame with two columns

    # ------------ identify habitat attribute names connected to Action Category/Type --------
    crosswalk_habitat_attribute_x = Habitat_Attribute_Names_Crosswalk[habitat_attribute_x]

    # ------------- identify Action Category and Action Type related to impaired habitat attribute
    pathways_habitat_attributes_x = Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame.loc[Crosswalk_Habitat_Attributes_and_Project_Categories_DataFrame['Habitat Attribute'].isin(crosswalk_habitat_attribute_x)]

    # ---------------------- Only get Projects developed for this specific reach ---------
    Projects_List_Specific_Reach_x = Project_List_Data.loc[Project_List_Data.Reach_UCSRB.isin([reach_name_x])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Type.isin(pathways_habitat_attributes_x['Action Type'])]

    # --------------------- List Projects by Action Type for this specific impaired habitat attribute -------------
    Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute = Projects_List_Specific_Reach_x.loc[Projects_List_Specific_Reach_x.Action_Category.isin(pathways_habitat_attributes_x['Action Category'])]

    #print(reach_name_x)
    #print(Projects_List_Specific_Reach_x.shape)
    #print(Projects_List_Specific_Reach_x_Habitat_Attribute.shape)

    # --------------- if there is no Projects in this specific reach --------------
    if Projects_List_Specific_Reach_x.shape[0] == 0:
        reaches_not_present_in_projects = reaches_not_present_in_projects.append(reach_habitat_attribute_data_frame)

    # -------- if there is Projects in this reach, but no Projects related to impaired habitat attribute -----
    elif Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]==0 or Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]==0:
        reaches_present_no_projects = reaches_present_no_projects.append(reach_habitat_attribute_data_frame)

    # ----------- if Projects for the impaired habitat attribute are present (based on Action Type or Action Category) ------------
    else:
        #print('-----------------reaches with projects for impaired habitat attribute-------------')
        #print(reach_habitat_attribute_data_frame)

        # ----------------- Add the habitat attribute name to the data frame ----------------
        # ------------ For Action Type data frame ---------
        reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
        reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute']
        reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.index
        Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

        # --------------- For Action Category data frame -------------
        reach_habitat_attribute_data_frame_for_output_x = reach_habitat_attribute_data_frame.append([reach_habitat_attribute_data_frame]*(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape[0]-1))
        reach_habitat_attribute_data_frame_for_output_x.columns = ['ReachName','Habitat_Attribute']
        reach_habitat_attribute_data_frame_for_output_x.index = Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.index
        Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute['Habitat_Attribute'] = reach_habitat_attribute_data_frame_for_output_x['Habitat_Attribute']

        # --------------- save output to individual Action Category/Action Type data frames ---------
        Projects_Action_Type_for_impaired_habitat_attributes = Projects_Action_Type_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
        Projects_Action_Category_for_impaired_habitat_attributes = Projects_Action_Category_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)

        # ------------- Save output to master data frame -------------------
        Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute)
        Projects_for_impaired_habitat_attributes = Projects_for_impaired_habitat_attributes.append(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute)
        #print(Projects_List_Action_Type_Specific_Reach_x_Habitat_Attribute.shape)
        #print(Projects_List_Action_Category_Specific_Reach_x_Habitat_Attribute.shape)

        
# ------------------------- Collapse duplicated projects to one project, list multiple habitat attributes ----------
project_data_frame = pd.DataFrame(Projects_for_impaired_habitat_attributes['ProjectID'])
duplicated_project_data_frame_x = project_data_frame.loc[project_data_frame.duplicated()].sort_values(by=['ProjectID'])

# ------- loop through projects to collapse duplicated projects ------- 
Projects_for_impaired_habitat_attributes_updated = pd.DataFrame([])
unique_Project_IDs_x = Projects_for_impaired_habitat_attributes.ProjectID.unique()
for projectID_x in unique_Project_IDs_x:
    
    # ---------- if a duplicated project --------
    if any(duplicated_project_data_frame_x.ProjectID.isin([projectID_x]) ): 
        Duplicated_Projects_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
        # ------- Habitat Attribute list for all the duplicated projects ----------
        Habitat_Attributes_duplicated_x = Duplicated_Projects_x.Habitat_Attribute
        Habitat_Attributes_duplicated_x = Habitat_Attributes_duplicated_x.unique()
        Habitat_Attributes_duplicated_x = str(np.stack(Habitat_Attributes_duplicated_x) )

        # ------------ Get a single row for all the duplicated projects ---------------
        row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])].iloc[0:1,]
        row_x.Habitat_Attribute = Habitat_Attributes_duplicated_x
        Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)

    # ------- if project ID is not duplicated ----------
    else:
        row_x = Projects_for_impaired_habitat_attributes.loc[Projects_for_impaired_habitat_attributes.ProjectID.isin([projectID_x])]
        Projects_for_impaired_habitat_attributes_updated = Projects_for_impaired_habitat_attributes_updated.append(row_x)

# ------------------------------ OUTPUT to an Excel spreadsheet  -------------------------  
Projects_for_impaired_habitat_attributes_updated.to_excel(output_location_Spring_Chinook, index = False)